Integrantes:

*   Daniel Perea Mercado
*   David Diaz Rodriguez
*   Nicolas Niño Valderrama
*   Valentina Jimenez Torres

# **Preparación del notebook**

___
## Cargue de librerias

In [ ]:
!pip install unidecode

In [ ]:
# Eliminar Warnings
import warnings
warnings.filterwarnings("ignore")

# Data
import pandas as pd
import numpy as np
from datetime import datetime
import unidecode
from scipy.stats import chi2_contingency
from sklearn.preprocessing import LabelEncoder

# Estadística
import math
import scipy.stats as stats
from math import pi
from scipy.stats import zscore

# Visualización
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp
import seaborn as sns
from matplotlib.patches import Patch
from IPython.display import Image

# Modelling
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error,  mean_absolute_error, mean_absolute_percentage_error, r2_score, classification_report, accuracy_score,confusion_matrix, ConfusionMatrixDisplay, f1_score
from sklearn import metrics, tree
import graphviz
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from sklearn.datasets import load_iris
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.datasets import make_classification, make_regression
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
from xgboost import XGBClassifier, XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

___
## Instalación del entorno de GitHub

In [ ]:
# Instalar git en Colab si no está instalado
!apt-get install git

In [ ]:
# Clonar el repositorio de GitHub
!git clone https://github.com/niconinov/Proyecto-Aula.git

___
## Descarga de las bases de datos en el notebook

In [ ]:
# Nombre de las bases de datos en GitHub
names =['algebra_vectorial', 'calculo_diferencial']

In [ ]:
# Lectura de los csv en GitHub
dfs = {}

for i in names:
  dfs[i] = pd.read_csv(f'/content/Proyecto-Aula/{i}.csv')

In [ ]:
# Creación de las bases de datos
df_alg = pd.DataFrame(dfs['algebra_vectorial'])
df_cal = pd.DataFrame(dfs['calculo_diferencial'].iloc[:, 1:])

In [ ]:
df_alg = pd.read_csv('/content/Proyecto-Aula/algebra_vectorial.csv', sep=';')
df_alg = df_alg.drop(df_alg.columns[0], axis=1)
df_alg.head()

#**PRIMERA ENTREGA**

___
## **Tratamiento de las bases de datos**

### CÁLCULO DIFERENCIAL

#### Exploración

In [ ]:
df_cal.shape

In [ ]:
# Respectivas variables ordinales en categorías ordinales

categorical_columns = ['madre_edu', 'padre_edu', 't_examen', 't_estudio', 'relacion_fam', 'tiempo_libre',
                        'salir_amigos', 'cons_alcohol_sem', 'cons_alcohol_finde', 'salud']

df_cal[categorical_columns] = df_cal[categorical_columns].astype('object')

In [ ]:
df_cal.info()

In [ ]:
df_cal.head()

In [ ]:
# Se guarda esta copia para cambiar el nombre de las columnas en el dataframe de álgebra
df_cal_og = df_cal.copy()

#### Datos duplicados

In [ ]:
df_cal.duplicated().sum()

In [ ]:
duplicadoscal = df_cal[df_cal.duplicated(keep=False)]
print(duplicadoscal)

In [ ]:
# Eliminación de los datos duplicados
df_cal = df_cal.drop_duplicates()

>En el DataFrame referente a los estudiantes de Calculo, se identificaron dos filas duplicadas, ubicadas en las posiciones 395 y 396. Debido a la naturaleza de los datos concideramos que no es posible que existan dos estudiantes con exactamente las misma caracteristicas, por lo cual tomamos la desición de eliminar estas filas duplicadas antes de continuar con el tratamiento y análisis de la base de datos.

####Valores nulos

In [ ]:
nuloscal = df_cal.isnull().sum()
for columna, cantidad in nuloscal.items():
    print(f"La columna '{columna}' tiene {cantidad} observaciones nulas.")

Se identifica y reporta la cantidad de valores nulos en cada columna del DataFrame, proporcionando una visión general de la cantidad de datos faltantes por columna, lo cual es útil para la limpieza y el análisis de datos. La salida del código ayudará a identificar las columnas que podrían necesitar atención adicional para manejar los valores nulos, ya sea mediante imputación, eliminación o alguna otra estrategia de tratamiento de datos faltantes. Para este caso, se puede evidenciar que en el DataFrame no se encuentran datos nulos.

#### Outliers

Revisamos el método gráfico para las variables con naturaleza numérica, las variables son: 'edad', 'faltas', 'ausencias', 'nota_01', 'nota_02', 'nota_03'.

In [ ]:
columnas_especificas = ['edad', 'faltas', 'ausencias', 'nota_01', 'nota_02', 'nota_03']

# Configuramos el número de columnas y filas
n_cols = 3
n_rows = math.ceil(len(columnas_especificas) / n_cols)

# Creamos la figura y los ejes
fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 6 * n_rows))

# Generamos los boxplots
for i, col in enumerate(columnas_especificas):
    row = i // n_cols
    col_idx = i % n_cols
    sns.boxplot(data=df_cal, x=col, ax=axes[row, col_idx])
    axes[row, col_idx].set_xlabel(col.capitalize())
    axes[row, col_idx].set_title(f'Boxplot de {col.capitalize()}')

# Eliminamos los ejes vacíos
for j in range(i + 1, n_rows * n_cols):
    row = j // n_cols
    col_idx = j % n_cols
    fig.delaxes(axes[row, col_idx])

# Ajustamos el layout para que no se solapen las etiquetas
plt.tight_layout()
plt.show()

Revisamos el z-core para aquellas variables con puntos fuera de la caja

In [ ]:
#Obtenemos el conteo de los valores únicos de cada variable para analizar los outliers
columnas = ['edad', 'faltas', 'ausencias', 'nota_02']

for col in columnas:
    conteo_valores = df_cal[col].value_counts().sort_index()
    print(f"Variable:")
    print(conteo_valores)
    print("---")

In [ ]:
for col in columnas:

    # Calcular z-score para cada columna y obtener su valor absoluto
    z_scores = zscore(df_cal[col])
    abs_z_scores = np.abs(z_scores)

    # Seleccionar los outliers usando un límite de 3
    outliers_zscore = df_cal[abs_z_scores > 3]

    # Contar el número de valores atípicos
    num_outliers = outliers_zscore[col].count()

    # Imprimir el nombre de la columna, el valor mínimo del outlier y el número de outliers
    print(f"Variable: {col}")
    print(f"Número de valores atípicos: {num_outliers}")
    print(f"Outlier mínimo (z-score): {outliers_zscore[col].min()}")
    print("---")

In [ ]:
# En el caso de la variable edad, considerando el tamaño relativamente pequeño del dataframe
# se decide imputar los 2 outliers con la mediana
z_scores = zscore(df_cal['edad'])
abs_z_scores = np.abs(z_scores)

outliers_zscore = df_cal[abs_z_scores > 3]

mediana = df_cal['edad'].median()
df_cal.loc[outliers_zscore.index, 'edad'] = mediana

print('La mediana de la variable edad es', mediana)
print("---")
print(df_cal['edad'].value_counts())

> En el caso de la variable faltas, no se consideran valores atípicos al resultado del código z-score pues entre el conteo para 2 faltas y el conteo para 3 faltas hay un sólo registro de diferencia. En este caso, se prefiere hacer una transformación a booleana creando las nuevas categorías 'No falta a clase' con los registros de cero faltas y otra categoría 'Falta a clase' con los registros de 1, 2 y 3 faltas.

In [ ]:
df_cal['faltas'].value_counts()

In [ ]:
# Aplicamos la recategorización para la variable faltas
def Transf(faltas):
    if faltas == 0:
        return 'No falta a clase'
    else:
        return 'Falta a clase'

# Aplicar la función a la columna 'ausencias'
df_cal['faltas'] = df_cal['faltas'].apply(Transf)
df_cal['faltas'].value_counts()

In [ ]:
# Aplicamos la recategorización para la variable ausencias
def Transf(ausencias):
    if ausencias == 0:
        return 'No falta a la universidad'
    else:
        return 'Falta a la universidad'

# Aplicar la función a la columna 'ausencias'
df_cal['ausencias'] = df_cal['ausencias'].apply(Transf)
df_cal['ausencias'].value_counts()

In [ ]:
# En el caso de la variable ausencias, considerando el tamaño relativamente pequeño del dataframe
# se decide imputar los 6 outliers con la mediana
# z_scores = zscore(df_cal['ausencias'])
# abs_z_scores = np.abs(z_scores)

# outliers_zscore = df_cal[abs_z_scores > 3]

# mediana = df_cal['ausencias'].median()
# df_cal.loc[outliers_zscore.index, 'ausencias'] = mediana

# print('La mediana para imputar en la variable ausencias es', mediana)
# print("---")
# print(df_cal['ausencias'].value_counts().sort_index())

>Finalmente para nota 02, el punto atípico del boxplot es la nota 0 que cuenta con 13 registros. Consideramos que en este caso y con apoyo del z-score que no muestra atípicos para la variable, no es necesario hacer tratamiento, pues brinda información relevante.

#### Análisis univariado

##### Variable objetivo

In [ ]:
df_cal['nota_03'].describe()

En la variable nota_03, la calificación promedio fue de 10-11, lo que sugiere que la mayoría de los estudiantes obtuvo notas cercanas a este valor. Sin embargo, hubo una gran variación en los resultados, la nota más baja fue 0, mientras que la más alta fue 20. Una cuarta parte de los estudiantes obtuvo menos de 8 puntos, mientras que la mitad logró una nota igual o menor a 11. Los estudiantes con mejores resultados, es decir, aquellos en el 25% superior, alcanzaron calificaciones de 14 o más.

In [ ]:
## Distribución de la variable
sns.set_style('darkgrid')

plt.figure(figsize=(5, 5))
sns.histplot(df_cal['nota_03'], bins=len(df_cal['nota_03'].unique()), kde=True, color='blue', edgecolor='black')

plt.title('Distribución de la variable objetivo nota_03', fontsize=14)
plt.xlabel('Nota 03', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)

plt.show()

In [ ]:
#retorno valores shapiro
statistic, p_value = stats.shapiro(df_cal['nota_03'])
#Imprimir estadistica de prueba
print("estadistica de prueba:", statistic)
#Imprimir p_value
print("p-value:", p_value)

Los resultados de la prueba de Shapiro para la variable nota_03 indican una estadística de prueba de 0.9287 y un p-valor extremadamente bajo (8.8359e-13). Dado que el p-valor es mucho menor que un nivel de significancia comúnmente utilizado (0.05), rechazamos la hipótesis nula de normalidad. Por lo tanto, podemos concluir que la variable nota_03 no sigue una distribución normal.

##### Análisis variables numéricas

###### Resumen estadístico

In [ ]:
# Resumen estadístico para las variables numéricas
columnas_especificas = ['edad', 'nota_01', 'nota_02']
df_cal[columnas_especificas].describe()

El análisis estadístico de las variables muestra que la edad promedio de los estudiantes es de 16 a 17 años, con poca variabilidad, y que la media de ausencias es 5-6, aunque con una alta dispersión, llegando hasta 28 ausencias en algunos casos. Las calificaciones promedio de nota_01 y nota_02 son 10-11, respectivamente, con una distribución relativamente simétrica en ambas, aunque nota_02 muestra una mayor dispersión y un valor mínimo de 0.

###### Distribuciones

In [ ]:
## Distribución de la variable
sns.set_style('darkgrid')

n_col = 2  # Número de columnas
n_row = (len(columnas_especificas) + 1) // 2  # Número de filas

fig, axes = plt.subplots(n_row, n_col, figsize=(12, 6))

# Aplanar la matriz de ejes para iterar fácilmente
axes = axes.flatten()

# Crear histogramas para cada variable
for i, columna in enumerate(columnas_especificas):
    sns.histplot(df_cal[columna], bins=len(df_cal[columna].unique()), kde=True, color='blue', edgecolor='black', ax=axes[i])
    axes[i].set_title(f'Histograma de {columna}', fontsize=14)
    axes[i].set_xlabel(columna, fontsize=12)
    axes[i].set_ylabel('Frecuencia', fontsize=12)

# Eliminar ejes vacíos si hay menos subgráficos que columnas
for j in range(len(columnas_especificas), len(axes)):
    fig.delaxes(axes[j])

# Ajustar el espacio entre las filas y columnas
plt.subplots_adjust(hspace=0.8, wspace=0.3)  # Aumenta el hspace para más espacio entre filas

plt.show()

###### Pruebas de normalidad

In [ ]:
for columna in columnas_especificas:
    statistic, p_value = stats.shapiro(df_cal[columna])

    print(f"\nVariable: {columna}")
    print("Estadística de prueba:", statistic)
    print("p-value:", p_value)

    if p_value > 0.05:
        print(f"La variable {columna} sigue una distribución normal (El p-value es mayor a 0.05).")
    else:
        print(f"La variable {columna} no sigue una distribución normal (El p-value es menor a 0.05).")

    print("\n-----------------------------------------")

No sólo la variable respuesta NO sigue una distribución normal, sino que además de esto, ninguna de las variables númericas lo sigue.

##### Análisis variables categóricas

###### **Variables categóricas con más de dos categorías**

####### Exploración

Para las variables categóricas consideramos que las categorías con menos del 5% se podrían considerar ruido, dependiendo del contexto.

In [ ]:
# Lista de columnas para graficar
columnas = [col for col in df_cal.select_dtypes(include=['object']).columns
                                   if len(df_cal[col].unique()) > 2]

# Número de gráficos
n = len(columnas)
n_cols = 2
n_rows = (n + 1) // n_cols  # Calcula el número de filas necesarias

fig, axes = plt.subplots(n_rows, n_cols, figsize=(14, 4 * n_rows))
axes = axes.flatten()  # Aplanar la matriz de ejes

for i, columna in enumerate(columnas):
    sizes = df_cal[columna].value_counts()
    labels = sizes.index
    counts = sizes.values

    # Crear el gráfico de dona sin etiquetas
    wedges, texts, autotexts = axes[i].pie(sizes, autopct='%1.1f%%', startangle=140, pctdistance=0.85)
    centre_circle = plt.Circle((0, 0), 0.70, fc='white')
    axes[i].add_artist(centre_circle)

    # Crear leyenda con conteos
    legend_labels = [f'{label} (Conteo: {count})' for label, count in zip(labels, counts)]
    axes[i].legend(wedges, legend_labels, title="Categorías", loc="upper left", bbox_to_anchor=(1, 1), fontsize='small')

    axes[i].set_title(f'Diagrama de Dona - {columna}')

# Eliminar ejes vacíos si hay menos gráficos que subplots
for ax in axes[n:]:
    ax.remove()

plt.tight_layout(rect=[0, 0, 0.75, 1])  # Ajustar el espacio para la leyenda
plt.show()

In [ ]:
print(columnas)

####### madre_edu y padre_edu

En el caso de estas variables, aunque la categoría 0 (ninguna educación) hace ruido al representar menos del 1% de los datos, consideramos que no es correto agruparla con ningún otra categoría, pues el contexto de no tener ningún tipo de educación es muy diferente a tener un mínimo grado pues esto puede influir en la habilidad de leer, escribir y tener conocimiento básico de matemáticas.

####### t_examen

En el caso de esta categoría, al tener una connotación importante tener mas de una hora con las demás opciones, no se considera viable reagrupar.

####### madre_trab

En el caso de esta variable, al no haber ningún tipo de relación entre los trabajos, tampoco es correcto pensar en reagrupaciones.

####### relacion_fam, tiempo_libre, salir_amigos, cons_alcohol_sem, cons_alcohol_finde, salud






En el caso de estas variables, calificadas desde 1 - muy bajo hasta 5 - muy alto, no es correcto reagrupar las categorías para darle tratamiento, ya que cada una representa una percepción y comportamiento en diferentes aspectos, los valores en cada variable tienen un significado específico dentro de su contexto, lo cual impide que se puedan tratar de manera uniforme.

###### **Variables boolenas**

>Para las variables booleanas hemos definido un umbral de 80-20. Donde las varibles que se distribuyan mayor de 80 y menor de 20, no serán significativas y se eliminarán del modelo.

In [ ]:
# Lista de columnas para graficar
columnas = [col for col in df_cal.select_dtypes(include=['object']).columns
            if len(df_cal[col].unique()) <= 2]

columnas_mayor_80 = []

# Evaluar cada columna y crear gráficos
n = len(columnas)
n_cols = 2
n_rows = (n + 1) // n_cols  # Calcula el número de filas necesarias

fig, axes = plt.subplots(n_rows, n_cols, figsize=(14, 4 * n_rows))
axes = axes.flatten()  # Aplanar la matriz de ejes

for i, columna in enumerate(columnas):
    sizes = df_cal[columna].value_counts(normalize=True) * 100
    if any(sizes > 80):
        columnas_mayor_80.append(columna)

    # Crear gráfico de dona para cada columna
    sizes = df_cal[columna].value_counts()
    labels = sizes.index
    counts = sizes.values

    # Crear el gráfico de dona
    wedges, texts, autotexts = axes[i].pie(sizes, autopct='%1.1f%%', startangle=140, pctdistance=0.85)
    centre_circle = plt.Circle((0, 0), 0.70, fc='white')
    axes[i].add_artist(centre_circle)

    # Crear leyenda con conteos
    legend_labels = [f'{label} (Conteo: {count})' for label, count in zip(labels, counts)]
    axes[i].legend(wedges, legend_labels, title="Categorías", loc="upper left", bbox_to_anchor=(1, 1), fontsize='small')

    axes[i].set_title(f'Diagrama de dona - {columna}')

# Eliminar ejes vacíos si hay menos gráficos que subplots
for ax in axes[n:]:
    ax.remove()

plt.tight_layout(rect=[0, 0, 0.75, 1])  # Ajustar el espacio para la leyenda
plt.show()

In [ ]:
print("Columnas con categorías mayores al 80%:")
print(columnas_mayor_80)

In [ ]:
df_cal = df_cal.drop(columns = columnas_mayor_80, axis=1)

#### Analisis Bivariado

##### Variables numéricas

###### Correlación y multicolinealidad

>Se revisa la correlación de las variables númericas con la variable a predecir

In [ ]:
corr_matrix_num = df_cal.select_dtypes(include=np.number).corr()
print("Correlaciones de las variables numericas con 'nota_03':")
print(corr_matrix_num["nota_03"].sort_values(ascending=False))

In [ ]:
threshold = 0.8 #este valor se utiliza para correlacion del 80 en adelante
filter_= np.abs(corr_matrix_num["nota_03"])> threshold
corr_features = corr_matrix_num.columns[filter_].tolist()
plt.figure(figsize=(12,8))
sns.heatmap(df_cal[corr_features].corr(),annot=True, cmap = 'Blues', fmt=".2f")
plt.title("Correlacion entre variables numericas (Threshold: {})".format(threshold))
plt.show()

In [ ]:
# Variables numéricas
sns.pairplot(df_cal[corr_features], diag_kind="kde")

>Por la alta correlación de nota_01 y nota_02 con la variable respuesta, se realiza la prueba de correlación entre ellas para revisar multicolinealidad.

In [ ]:
correlacion_nota_01_02 = df_cal["nota_01"].corr(df_cal["nota_02"])
print("La correlación entre nota_01 y nota_02 es:", correlacion_nota_01_02)

>Al encontrar multicolinealidad entre ellas, se decide eliminar nota_01, pues es la que menor correlación tiene con la variable respuesta..

In [ ]:
df_cal = df_cal.drop(['nota_01'], axis=1)

##### Variables categóricas con más de dos categorías

###### Correlaciones con matriz de Cramér's V

>Se revisa la correlación de estas variables categóricas entre sí.
Cramér's V usa la tabla de contingencia para calcular el estadístico Chi-cuadrado, que luego se convierte en una medida de la fuerza de la asociación entre las variables categóricas. El estadístico de pruba V va de 0 (sin asociación) a 1 (asociación perfecta).

In [ ]:
def cramers_v(x, y):
    # Crear una tabla de contingencia
    contingency_table = pd.crosstab(x, y)
    # Calcular el estadístico Chi-Cuadrado
    chi2_stat, _, _, _ = chi2_contingency(contingency_table)
    # Obtener el número de observaciones
    n = contingency_table.sum().sum()
    # Número de filas y columnas
    k = min(contingency_table.shape) - 1
    # Calcular Cramer's V
    return np.sqrt(chi2_stat / (n * k))

In [ ]:
# Lista de variables categóricas
categorical_vars = [col for col in df_cal.select_dtypes(include=['object']).columns
                                   if len(df_cal[col].unique()) > 2]

# Crear un DataFrame para almacenar los resultados
cramers_v_results = pd.DataFrame(index=categorical_vars, columns=categorical_vars)

# Calcular Cramér's V para cada par de variables categóricas
for var1 in categorical_vars:
    for var2 in categorical_vars:
        if var1 != var2:
            cramers_v_value = cramers_v(df_cal[var1], df_cal[var2])
            cramers_v_results.loc[var1, var2] = cramers_v_value
        else:
            # Para la diagonal (mismo par de variables), podemos poner NaN o 0
            cramers_v_results.loc[var1, var2] = np.nan

print("Matriz de Cramér's V entre variables categóricas:")
print(cramers_v_results)

>En el caso de las variables categóricas ordinales, no encontramos problemas de colinealidad o multicolinealidad, considerando correlacines entre ellas con un nivel de significancia por encima del 80%.

##### Variables categóricas booleanas

###### Correlaciones con matriz de Cramér's V

>Se revisa la correlación de estas variables categóricas entre sí.
Cramér's V usa la tabla de contingencia para calcular el estadístico Chi-cuadrado, que luego se convierte en una medida de la fuerza de la asociación entre las variables categóricas. El estadístico de pruba V va de 0 (sin asociación) a 1 (asociación perfecta).

In [ ]:
# Lista de variables categóricas
categorical_vars = [col for col in df_cal.select_dtypes(include=['object']).columns
                                   if len(df_cal[col].unique() == 2)]

# Crear un DataFrame para almacenar los resultados
cramers_v_results = pd.DataFrame(index=categorical_vars, columns=categorical_vars)

# Calcular Cramér's V para cada par de variables categóricas
for var1 in categorical_vars:
    for var2 in categorical_vars:
        if var1 != var2:
            cramers_v_value = cramers_v(df_cal[var1], df_cal[var2])
            cramers_v_results.loc[var1, var2] = cramers_v_value
        else:
            # Para la diagonal (mismo par de variables), podemos poner NaN o 0
            cramers_v_results.loc[var1, var2] = np.nan

print("Matriz de Cramér's V entre variables categóricas:")
print(cramers_v_results)

>En el caso de las variables categóricas booleanas, no encontramos problemas de colinealidad o multicolinealidad, considerando correlacines entre ellas con un nivel de significancia por encima del 80%.

#### Transformaciones

>Revisamos los valores únicos dentro de cada variable de manera gráfica

In [ ]:
# Se valida que variables no pueden ser booleanas
columnas_categoricas_no_dummies = [col for col in df_cal.select_dtypes(include=['object']).columns
                                   if len(df_cal[col].unique()) > 2]

print("Columnas con más de dos valores únicos:")
print(columnas_categoricas_no_dummies)

In [ ]:
# Conversión de variables categóricas con dos valores únicos en variables dummies
df_cal = pd.get_dummies(df_cal,
                        columns=[col for col in df_cal.select_dtypes(include=['object']).columns
                                 if col not in columnas_categoricas_no_dummies],
                        drop_first=True)

df_cal.info()

In [ ]:
# Para las variables categóricas con más de dos valores únicos se trabaja el método One-Hot para
# transformarlas en un conjunto de columnas binarias que indiquen la presencia o ausencia de la categoría

columnas_originales = df_cal.columns.tolist()
df_cal = pd.get_dummies(df_cal, columns=columnas_categoricas_no_dummies)
columnas_resultantes = df_cal.columns.tolist()

# Obtener las nuevas columnas generadas por pd.get_dummies() para mas adelante
nuevas_columnas = list(set(columnas_resultantes) - set(columnas_originales))
print(nuevas_columnas)

In [ ]:
# Se normalizan las columnas númericas dividiendo cada valor por el máximo valor en la columna respectiva,
# de forma que los valores queden en un rango entre 0 y 1, exceptuando la variable a precedir 'nota_03'.
numeric_columns = []

for col in df_cal.select_dtypes(include=np.number).columns:
    numeric_columns.append(col)
    if col != 'nota_03':
        df_cal[col] = df_cal[col] / df_cal[col].max()

df_cal[numeric_columns].head()

In [ ]:
#Se normaliza la variable a predecir (opcional)
# df_cal['nota_03'] = df_cal['nota_03']/df_cal['nota_03'].max()
# df_cal['nota_03'].head()

In [ ]:
df_cal.head()

In [ ]:
df_cal.info()

#### Análisis univariado VARIABLES NUEVAS

##### **Variables boolenas**

>Para las variables booleanas hemos definido un umbral de 80-20. Donde las varibles que se distribuyan mayor de 80 y menor de 20, no serán significativas y se eliminarán del modelo.

In [ ]:
# Lista de columnas para graficar
nuevas_columnas.sort()
columnas = nuevas_columnas

columnas_mayor_80 = []

# Evaluar cada columna y crear gráficos
n = len(columnas)
n_cols = 2
n_rows = (n + 1) // n_cols  # Calcula el número de filas necesarias

fig, axes = plt.subplots(n_rows, n_cols, figsize=(14, 4 * n_rows))
axes = axes.flatten()  # Aplanar la matriz de ejes

for i, columna in enumerate(columnas):
    sizes = df_cal[columna].value_counts(normalize=True) * 100
    if any(sizes > 80):
        columnas_mayor_80.append(columna)

    # Crear gráfico de dona para cada columna
    sizes = df_cal[columna].value_counts()
    labels = sizes.index
    counts = sizes.values

    # Crear el gráfico de dona
    wedges, texts, autotexts = axes[i].pie(sizes, autopct='%1.1f%%', startangle=140, pctdistance=0.85)
    centre_circle = plt.Circle((0, 0), 0.70, fc='white')
    axes[i].add_artist(centre_circle)

    # Crear leyenda con conteos
    legend_labels = [f'{label} (Conteo: {count})' for label, count in zip(labels, counts)]
    axes[i].legend(wedges, legend_labels, title="Categorías", loc="upper left", bbox_to_anchor=(1, 1), fontsize='small')

    axes[i].set_title(f'Diagrama de dona - {columna}')

# Eliminar ejes vacíos si hay menos gráficos que subplots
for ax in axes[n:]:
    ax.remove()

plt.tight_layout(rect=[0, 0, 0.75, 1])  # Ajustar el espacio para la leyenda
plt.show()

In [ ]:
print("Columnas con categorías mayores al 80%:")
columnas_mayor_80.sort()
print(columnas_mayor_80)

In [ ]:
df_cal = df_cal.drop(columns = ['razon_otro', 't_examen_3', 't_examen_4'], axis=1)

##### **Ultimas transformaciónes**

>Al analizar las nuevas columnas resultantes de la dummización de las variables categóricas con más de dos valores únicos, identificamos que algunas categorías presentaban una baja representación, lo que podría influir negativamente en la capacidad predictiva del modelo. Para mitigar este efecto, decidimos agrupar valores con características similares dentro de estas variables categóricas. Este agrupamiento nos permitió mejorar la proporción de los datos en cada categoría, generando clases más equilibradas y con mayor representatividad. Además, renombramos estas nuevas variables agrupadas.

In [ ]:
df_cal1 = df_cal.copy()

In [ ]:
# Combinamos las columnas 4 y 5 de la variable cons_alcohol_finde_4
df_cal1['cons_alcohol_finde_4'] = df_cal1['cons_alcohol_finde_4'] | ~df_cal1['cons_alcohol_finde_4'] & df_cal1['cons_alcohol_finde_5']
df_cal1 = df_cal1.drop('cons_alcohol_finde_5', axis = 1)

In [ ]:
# Combinamos las columnas de la variable cons_alcohol_sem dejando solo 1 categoría cons_alcohol_sem_reg que corresponde a
# los niveles 3,4 y 5 de la variable original y el false sería un consumo esporádico con los niveles 1 y 2
df_cal1['cons_alcohol_sem_reg'] = (df_cal1['cons_alcohol_sem_3'] | ~df_cal1['cons_alcohol_sem_3'] & df_cal1['cons_alcohol_sem_4']) | ~df_cal1['cons_alcohol_sem_3'] & ~df_cal1['cons_alcohol_sem_4'] & df_cal1['cons_alcohol_sem_5']
df_cal1 = df_cal1.drop(columns = ['cons_alcohol_sem_3', 'cons_alcohol_sem_4', 'cons_alcohol_sem_5'], axis = 1)

In [ ]:
# Combinamos las columnas 0 y 1 de la variable padre_edu
df_cal1['padre_edu_1'] = df_cal1['padre_edu_1'] | ~df_cal1['padre_edu_1'] & df_cal1['padre_edu_0']
df_cal1 = df_cal1.drop('padre_edu_0', axis = 1)

In [ ]:
# Combinamos las columnas 0 y 1 de la variable madre_edu
df_cal1['madre_edu_1'] = df_cal1['madre_edu_1'] | ~df_cal1['madre_edu_1'] & df_cal1['madre_edu_0']
df_cal1 = df_cal1.drop('madre_edu_0', axis = 1)

In [ ]:
# Incluimos la opción salud en otros para la variable madre_trab
df_cal1['madre_trab_otro'] = df_cal1['madre_trab_otro'] | ~df_cal1['madre_trab_otro'] & df_cal1['madre_trab_salud']
df_cal1 = df_cal1.drop('madre_trab_salud', axis = 1)

In [ ]:
# Incluimos la opción en casa en otros para la variable padre_trab
df_cal1['padre_trab_otro'] = df_cal1['padre_trab_otro'] | ~df_cal1['padre_trab_otro'] & df_cal1['padre_trab_en_casa']
df_cal1 = df_cal1.drop('padre_trab_en_casa', axis = 1)

In [ ]:
# Incluimos la opción salud y profesor en servicios para la variable padre_trab, entendiendo servicios como trabajos
# del sector publico
df_cal1['padre_trab_servicios'] = (df_cal1['padre_trab_servicios'] | ~df_cal1['padre_trab_servicios'] & df_cal1['padre_trab_profesor']) | ~df_cal1['padre_trab_servicios'] & ~df_cal1['padre_trab_profesor'] & df_cal1['padre_trab_salud']
df_cal1 = df_cal1.drop(columns = ['padre_trab_profesor', 'padre_trab_salud'], axis = 1)

In [ ]:
#Reducimos las opciones de relacion_fam de 1,2,3,4 y 5 a regular (1,2 y 3), buena (4) y excelente (5)
df_cal1['relacion_fam_reg'] = (df_cal1['relacion_fam_1'] | ~df_cal1['relacion_fam_1'] & df_cal1['relacion_fam_2']) | ~df_cal1['relacion_fam_1'] & ~df_cal1['relacion_fam_2'] & df_cal1['relacion_fam_3']
df_cal1 = df_cal1.drop(columns = ['relacion_fam_1', 'relacion_fam_2', 'relacion_fam_3'], axis = 1)

In [ ]:
# Renombrar la variable
df_cal1 = df_cal1.rename(columns={'relacion_fam_4': 'relacion_fam_buena'})

In [ ]:
# Renombrar la variable
df_cal1 = df_cal1.rename(columns={'relacion_fam_5': 'relacion_fam_exc'})

In [ ]:
#Reducimos las opciones de salir_amigos de 1,2,3,4 y 5 a bajo (1 y 2), medio (3) y alto (4 y 5)
df_cal1['salir_amigos_bajo'] = df_cal1['salir_amigos_2'] | ~df_cal1['salir_amigos_2'] & df_cal1['salir_amigos_1']
df_cal1 = df_cal1.drop(columns = ['salir_amigos_1', 'salir_amigos_2'], axis = 1)

In [ ]:
# Renombrar la variable
df_cal1 = df_cal1.rename(columns={'salir_amigos_3': 'salir_amigos_medio'})

In [ ]:
# Combinamos las columnas con el operador | y ~
df_cal1['salir_amigos_alto'] = df_cal1['salir_amigos_4'] | ~df_cal1['salir_amigos_4'] & df_cal1['salir_amigos_5']
df_cal1 = df_cal1.drop(columns = ['salir_amigos_4', 'salir_amigos_5'], axis = 1)

In [ ]:
#Reducimos las opciones de salud de 1,2,3,4 y 5 a mala (1 y 2), regular (3) y alta (4 y 5)
df_cal1['salud_mala'] = df_cal1['salud_1'] | ~df_cal1['salud_1'] & df_cal1['salud_2']
df_cal1 = df_cal1.drop(columns = ['salud_1', 'salud_2'], axis = 1)

In [ ]:
# Renombrar la variable
df_cal1 = df_cal1.rename(columns={'salud_3': 'salud_regular'})

In [ ]:
# Combinamos las columnas con el operador | y ~
df_cal1['salud_alta'] = df_cal1['salud_4'] | ~df_cal1['salud_4'] & df_cal1['salud_5']
df_cal1 = df_cal1.drop(columns = ['salud_4', 'salud_5'], axis = 1)

In [ ]:
# Combinamos las columnas 3 y 4 de la variable t_estudio
df_cal1['t_estudio_3'] = df_cal1['t_estudio_3'] | ~df_cal1['t_estudio_3'] & df_cal1['t_estudio_4']
df_cal1 = df_cal1.drop('t_estudio_4', axis = 1)

In [ ]:
#Reducimos las opciones de salud de 1,2,3,4 y 5 a vajo (1 y 2), medio (3) y alto (4 y 5)
df_cal1['tiempo_libre_bajo'] = df_cal1['tiempo_libre_1'] | ~df_cal1['tiempo_libre_1'] & df_cal1['tiempo_libre_2']
df_cal1 = df_cal1.drop(columns = ['tiempo_libre_1', 'tiempo_libre_2'], axis = 1)

In [ ]:
# Renombrar la variable
df_cal1 = df_cal1.rename(columns={'tiempo_libre_3': 'tiempo_libre_medio'})

In [ ]:
# Combinamos las columnas con el operador | y ~
df_cal1['tiempo_libre_alto'] = df_cal1['tiempo_libre_4'] | ~df_cal1['tiempo_libre_4'] & df_cal1['tiempo_libre_5']
df_cal1 = df_cal1.drop(columns = ['tiempo_libre_4', 'tiempo_libre_5'], axis = 1)

In [ ]:
columnas_resultantes = df_cal1.columns.tolist()

# Obtener las nuevas columnas generadas por pd.get_dummies() para mas adelante
nuevas_columnas = list(set(columnas_resultantes) - set(columnas_originales))

# Lista de columnas para graficar
nuevas_columnas.sort()
columnas = nuevas_columnas

columnas_mayor_80 = []

# Evaluar cada columna y crear gráficos
n = len(columnas)
n_cols = 2
n_rows = (n + 1) // n_cols  # Calcula el número de filas necesarias

fig, axes = plt.subplots(n_rows, n_cols, figsize=(14, 4 * n_rows))
axes = axes.flatten()  # Aplanar la matriz de ejes

for i, columna in enumerate(columnas):
    sizes = df_cal1[columna].value_counts(normalize=True) * 100
    if any(sizes > 80):
        columnas_mayor_80.append(columna)

    # Crear gráfico de dona para cada columna
    sizes = df_cal1[columna].value_counts()
    labels = sizes.index
    counts = sizes.values

    # Crear el gráfico de dona
    wedges, texts, autotexts = axes[i].pie(sizes, autopct='%1.1f%%', startangle=140, pctdistance=0.85)
    centre_circle = plt.Circle((0, 0), 0.70, fc='white')
    axes[i].add_artist(centre_circle)

    # Crear leyenda con conteos
    legend_labels = [f'{label} (Conteo: {count})' for label, count in zip(labels, counts)]
    axes[i].legend(wedges, legend_labels, title="Categorías", loc="upper left", bbox_to_anchor=(1, 1), fontsize='small')

    axes[i].set_title(f'Diagrama de dona - {columna}')

# Eliminar ejes vacíos si hay menos gráficos que subplots
for ax in axes[n:]:
    ax.remove()

plt.tight_layout(rect=[0, 0, 0.75, 1])  # Ajustar el espacio para la leyenda
plt.show()

>Para las variables booleanas hemos definido un umbral de 80-20. Donde las varibles que se distribuyan mayor de 80 y menor de 20, no serán significativas y se eliminarán del modelo.

In [ ]:
print("Columnas con categorías mayores al 80%:")
columnas_mayor_80.sort()
print(columnas_mayor_80)

In [ ]:
df_cal1 = df_cal1.drop(columns = columnas_mayor_80, axis=1)

In [ ]:
df_cal1.shape

> Al final de la exploración y el preprocesado de la base de datos, el data frame queda con 51 columnas y 395 filas

#### Modelo de regresión

In [ ]:
# Definir las columnas para el DataFrame
columns = ['DF', 'Modelo', 'Accuracy', 'MSE', 'RMSE', 'MAE', 'MAPE', 'R2', 'R2_adjusted']

# Inicializar un DataFrame vacío
df_resultados = pd.DataFrame(columns=columns)

##### Regresión lineal


In [ ]:
# definir gráfica
fig = px.histogram(df_cal1, x = 'nota_03', nbins = 50, width = 650, height = 500, title = '<b>Histograma nota 3<b>')

# agregar detalles
fig.update_layout(
    xaxis_title = '<b>Nota<b>',
    yaxis_title = '<b>Frecuencia<b>',
    template = 'simple_white',
    title_x = 0.5,
    barmode='overlay')
fig.update_traces(opacity=0.75)

fig.show()

In [ ]:
X_cal = df_cal1.drop(["nota_03"], axis=1)
y_cal = df_cal1["nota_03"]
# Tranformación logarítmica
y_log = np.log(y_cal + 1)
# Separar los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_cal, y_log, test_size=0.3, random_state=59)
# Inicializar y entrenar el modelo GBM
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train) # .fit: entrenar el modelo
# Realizar predicciones
lr_pred = lr_reg.predict(X_test)
DF = 'df_cal, orig'
Modelo = 'Test_original'
# Evaluación del modelo (por ejemplo, precisión)
accuracy = lr_reg.score(X_test, y_test)
print("Accuracy:", accuracy)
# Metricas
MSE = mean_squared_error(y_test, lr_pred)
print("MSE: %.2f" % MSE)
RMSE = math.sqrt(mean_squared_error(y_test, lr_pred))
print("RMSE %.2f" % RMSE)
MAE = mean_absolute_error(y_test, lr_pred)
print("MAE: %.2f" % MAE)
MAPE = mean_absolute_percentage_error(y_test, lr_pred)
print("MAPE: %.2f" % MAPE)
r2 = r2_score(y_test, lr_pred)
print("R2: %.2f" % r2)
# R2 ajustado
adj_r2 = 1 - (1-r2)*(len(y_test )-1)/(len(y_test)-X_test.shape[1]-1)
print("R2-adjusted: %.2f" % adj_r2)

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [accuracy],
    'MSE': [MSE],
    'RMSE': [RMSE],
    'MAE': [MAE],
    'MAPE': [MAPE],
    'R2': [r2],
    'R2_adjusted': [adj_r2]
})

# Concatenar con el DataFrame original
df_resultados = pd.concat([df_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(df_resultados)

In [ ]:
lr_pred = lr_reg.predict(X_train)

In [ ]:
DF = 'df_cal, orig'
Modelo = 'Train_original'
# Metricas
accuracy = lr_reg.score(X_train, y_train)
print("Accuracy: %.2f" % accuracy)
MSE = mean_squared_error(y_train, lr_pred)
print("MSE: %.2f" % MSE)
RMSE = math.sqrt(mean_squared_error(y_train, lr_pred))
print("RMSE %.2f" % RMSE)
MAE = mean_absolute_error(y_train, lr_pred)
print("MAE: %.2f" % MAE)
MAPE = mean_absolute_percentage_error(y_train, lr_pred)
print("MAPE: %.2f" % MAPE)
r2 = r2_score(y_train, lr_pred)
print("R2: %.2f" % r2)
# R2 ajustado
adj_r2 = 1 - (1-r2)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
print("R2-adjusted: %.2f" % adj_r2)

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [accuracy],
    'MSE': [MSE],
    'RMSE': [RMSE],
    'MAE': [MAE],
    'MAPE': [MAPE],
    'R2': [r2],
    'R2_adjusted': [adj_r2]
})

# Concatenar con el DataFrame original
df_resultados = pd.concat([df_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(df_resultados)

> Luego de calcular las métricas del moldelo, le realizamos una transformación inversa a las predicciones y a los valores reales para una mejor visualización.

In [ ]:
# Aplicar la transformación logarítmica inversa
y_test_inv = np.exp(y_test) - 1
lr_pred_inv = np.exp(lr_pred) - 1

In [ ]:
y_test_inv.head()

In [ ]:
print(lr_pred_inv)

### ÁLGEBRA VECTORIAL

#### Exploración

In [ ]:
df_alg.shape

In [ ]:
df_alg.info()

In [ ]:
df_alg.head()

___
Al revisar la dimesión del data frame de Algebra Vectorial notamos que toda la información esta condensada en una única columna que esta compuesta por las demás variables separadas por ";" por lo que debemos dividirlas para trabajar con ellas.

In [ ]:
# Separación de las columnas
#df_alg1 = df_alg.iloc[:, 0].str.split(';', expand=True)
# Eliminar la primera columna que actúa como índice
#df_alg1 = df_alg1.drop(df_alg1.columns[0], axis=1)
#df_alg1.shape

In [ ]:
df_alg1 = df_alg

In [ ]:
df_alg1.head()

Además, cambiamos los nombres de las columnas para mejor entendimiento.

In [ ]:
# Extraer los nombres de las columnas de df_cal_og
col_name = df_cal_og.columns.tolist()

# Reemplazar los nombres de las columnas en df_agl1 con los extraídos de df_cal_og
#df_alg1.columns = col_name

In [ ]:
df_alg1.head()

In [ ]:
df_alg1.info()

#### Datos duplicados

In [ ]:
df_alg1.duplicated().sum()

In [ ]:
duplicadosalg = df_alg1[df_alg1.duplicated(keep=False)]
print(duplicadosalg)

In [ ]:
df_alg1 = df_alg1.drop_duplicates()

#### Valores nulos

In [ ]:
nulosalg1 = df_alg1.isnull().sum()

for columna, cantidad in nulosalg1.items():
    print(f"La columna '{columna}' tiene {cantidad} observaciones nulas.")

#### Resumen estadístico

In [ ]:
pd.set_option('display.float_format', lambda x: '{:.2f}'.format(x))
df_alg1.describe()

#### Outliers

Revisamos el método gráfico para las variables con naturaleza numérica, las variables son: 'edad', 'faltas', 'ausencias', 'nota_01', 'nota_02', 'nota_03'.

In [ ]:
columnas_especificas = ['edad', 'faltas', 'ausencias', 'nota_01', 'nota_02', 'nota_03']

# Configuramos el número de columnas y filas
n_cols = 3
n_rows = math.ceil(len(columnas_especificas) / n_cols)

# Creamos la figura y los ejes
fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 6 * n_rows))

# Generamos los boxplots
for i, col in enumerate(columnas_especificas):
    row = i // n_cols
    col_idx = i % n_cols
    sns.boxplot(data=df_alg1, x=col, ax=axes[row, col_idx])
    axes[row, col_idx].set_xlabel(col.capitalize())
    axes[row, col_idx].set_title(f'Boxplot de {col.capitalize()}')

# Eliminamos los ejes vacíos
for j in range(i + 1, n_rows * n_cols):
    row = j // n_cols
    col_idx = j % n_cols
    fig.delaxes(axes[row, col_idx])

# Ajustamos el layout para que no se solapen las etiquetas
plt.tight_layout()
plt.show()

Revisamos el z-core para aquellas variables con puntos fuera de la caja

In [ ]:
#Obtenemos el conteo de los valores únicos de cada variable para analizar los outliers
columnas = ['edad', 'faltas', 'ausencias', 'nota_01', 'nota_02', 'nota_03']

for col in columnas:
    conteo_valores = df_alg1[col].value_counts().sort_index()
    print(f"Variable:")
    print(conteo_valores)
    print("---")

In [ ]:
for col in columnas:

    # Calcular z-score para cada columna y obtener su valor absoluto
    z_scores = zscore(df_alg1[col])
    abs_z_scores = np.abs(z_scores)

    # Seleccionar los outliers usando un límite de 3
    outliers_zscore = df_alg1[abs_z_scores > 3]

    # Contar el número de valores atípicos
    num_outliers = outliers_zscore[col].count()

    # Imprimir el nombre de la columna, el valor mínimo del outlier y el número de outliers
    print(f"Variable: {col}")
    print(f"Número de valores atípicos: {num_outliers}")
    print(f"Outlier mínimo (z-score): {outliers_zscore[col].min()}")
    print("---")

In [ ]:
# En el caso de la variable faltas, no se consideran valores atípicos al resultado del código z-score pues
# entre el conteo para 2 faltas y el conteo para 3 faltas hay un sólo registro de diferencia. En este caso,
# se prefiere hacer una transformación a booleana creando las nuevas categorías 'No falta a clase' con los
# registros de cero faltas y otra categoría 'Falta a clase' con los registros de 1, 2 y 3 faltas.
df_alg1['faltas'].value_counts()

In [ ]:
# Aplicamos la recategorización para la variable faltas
def Transf(faltas):
    if faltas == 0:
        return 'No falta a clase'
    else:
        return 'Falta a clase'

# Aplicar la función a la columna 'ausencias'
df_alg1['faltas'] = df_alg1['faltas'].apply(Transf)
df_alg1['faltas'].value_counts()

In [ ]:
# Aplicamos la recategorización para la variable ausencias
def Transf(ausencias):
    if ausencias == 0:
        return 'No se asuenta a la U'
    else:
        return 'Se ausenta a la U'

# Aplicar la función a la columna 'ausencias'
df_alg1['ausencias'] = df_alg1['ausencias'].apply(Transf)
df_alg1['ausencias'].value_counts()

En el caso de las notas, el z-score nos está mostrando la nota 0 como límite de outliers, pero hemos considerado que es un valor que representa información relevante.

#### Análisis univariado

##### Variable objetivo

In [ ]:
## Distribución de la variable
sns.set_style('darkgrid')

plt.figure(figsize=(5, 5))
sns.histplot(df_alg1['nota_03'], bins=len(df_alg1['nota_03'].unique()), kde=True, color='blue', edgecolor='black')

plt.title('Distribución de la variable objetivo nota_03', fontsize=14)
plt.xlabel('Nota 03', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)

plt.show()

In [ ]:
#retorno valores shapiro
statistic, p_value = stats.shapiro(df_alg1['nota_03'])
#Imprimir estadistica de prueba
print("estadistica de prueba:", statistic)
#Imprimir p_value
print("p-value:", p_value)

Conclusion

##### Análisis variable numéricas

###### Resumen estadístico

In [ ]:
# Resumen estadístico para las variables numéricas
columnas_especificas = ['nota_01', 'nota_02']
df_alg1[columnas_especificas].describe()

El análisis estadístico de las variables muestra que la edad promedio de los estudiantes es de 16 a 17 años, con poca variabilidad, y que la media de ausencias es 3-4, aunque con una alta dispersión, llegando hasta 32 ausencias en algunos casos. Las calificaciones promedio de nota_01 y nota_02 son 11-12, con una distribución relativamente simétrica en ambas, aunque nota_02 muestra una mayor dispersión.

###### Distribuciones

In [ ]:
## Distribución de la variable
sns.set_style('darkgrid')

n_col = 2  # Número de columnas
n_row = (len(columnas_especificas) + 1) // 2  # Número de filas

fig, axes = plt.subplots(n_row, n_col, figsize=(12, 6))

# Aplanar la matriz de ejes para iterar fácilmente
axes = axes.flatten()

# Crear histogramas para cada variable
for i, columna in enumerate(columnas_especificas):
    sns.histplot(df_alg1[columna], bins=len(df_alg1[columna].unique()), kde=True, color='blue', edgecolor='black', ax=axes[i])
    axes[i].set_title(f'Histograma de {columna}', fontsize=14)
    axes[i].set_xlabel(columna, fontsize=12)
    axes[i].set_ylabel('Frecuencia', fontsize=12)

# Eliminar ejes vacíos si hay menos subgráficos que columnas
for j in range(len(columnas_especificas), len(axes)):
    fig.delaxes(axes[j])

# Ajustar el espacio entre las filas y columnas
plt.subplots_adjust(hspace=0.8, wspace=0.3)  # Aumenta el hspace para más espacio entre filas

plt.show()

##### Pruebas de normalidad

In [ ]:
for columna in columnas_especificas:
    statistic, p_value = stats.shapiro(df_alg1[columna])

    print(f"\nVariable: {columna}")
    print("Estadística de prueba:", statistic)
    print("p-value:", p_value)

    if p_value > 0.05:
        print(f"La variable {columna} sigue una distribución normal (El p-value es mayor a 0.05).")
    else:
        print(f"La variable {columna} no sigue una distribución normal (El p-value es menor a 0.05).")

    print("\n-----------------------------------------")

No sólo la variable respuesta NO sigue una distribución normal, sino que además de esto, ninguna de las variables númericas lo sigue.

##### Análisis variables categóricas

##### **Variables categóricas con más de dos categorías**

###### Exploración

In [ ]:
# Lista de columnas para graficar
columnas = [col for col in df_alg1.select_dtypes(include=['object']).columns
                                   if len(df_alg1[col].unique()) > 2]

# Número de gráficos
n = len(columnas)
n_cols = 2
n_rows = (n + 1) // n_cols  # Calcula el número de filas necesarias

fig, axes = plt.subplots(n_rows, n_cols, figsize=(14, 4 * n_rows))
axes = axes.flatten()  # Aplanar la matriz de ejes

for i, columna in enumerate(columnas):
    sizes = df_alg1[columna].value_counts()
    labels = sizes.index
    counts = sizes.values

    # Crear el gráfico de dona sin etiquetas
    wedges, texts, autotexts = axes[i].pie(sizes, autopct='%1.1f%%', startangle=140, pctdistance=0.85)
    centre_circle = plt.Circle((0, 0), 0.70, fc='white')
    axes[i].add_artist(centre_circle)

    # Crear leyenda con conteos
    legend_labels = [f'{label} (Conteo: {count})' for label, count in zip(labels, counts)]
    axes[i].legend(wedges, legend_labels, title="Categorías", loc="upper left", bbox_to_anchor=(1, 1), fontsize='small')

    axes[i].set_title(f'Diagrama de dona - {columna}')

# Eliminar ejes vacíos si hay menos gráficos que subplots
for ax in axes[n:]:
    ax.remove()

plt.tight_layout(rect=[0, 0, 0.75, 1])  # Ajustar el espacio para la leyenda
plt.show()

###### madre_edu y padre_edu

En el caso de estas variables, aunque la categoría 0 (ninguna educación) hace ruido al representar menos del 1% de los datos, consideramos que no es correto agruparla con ningún otra categoría, pues el contexto de no tener ningún tipo de educación es muy diferente a tener un mínimo grado pues esto puede influir en la habilidad de leer, escribir y tener conocimiento básico de matemáticas.

###### t_examen

En el caso de esta categoría, al tener una connotación importante tener mas de una hora con las demás opciones, no se considera viable reagrupar.

###### relacion_fam, tiempo_libre, salir_amigos, cons_alcohol_sem, cons_alcohol_finde, salud






En el caso de estas variables, calificadas desde 1 - muy bajo hasta 5 - muy alto, no es correcto reagrupar las categorías para darle tratamiento, ya que cada una representa una percepción y comportamiento en diferentes aspectos, los valores en cada variable tienen un significado específico dentro de su contexto, lo cual impide que se puedan tratar de manera uniforme.

###### padre_trab

En el caso de esta variable, al no haber ningún tipo de relación entre los trabajos, tampoco es correcto pensar en reagrupaciones.

Para las variables categóricas consideramos que las categorías con menos del 5% se podrían considerar ruido, dependiendo del contexto.

##### **Variables boolenas**

Para las variables booleanas hemos definido un umbral de 80-20. Donde las varibles que se distribuyan mayor de 80 y menor de 20, no serán significativas y se eliminarán del modelo.

In [ ]:
# Lista de columnas para graficar
columnas = [col for col in df_alg1.select_dtypes(include=['object']).columns
            if len(df_alg1[col].unique()) <= 2]

columnas_mayor_80 = []

# Evaluar cada columna y crear gráficos
n = len(columnas)
n_cols = 2
n_rows = (n + 1) // n_cols  # Calcula el número de filas necesarias

fig, axes = plt.subplots(n_rows, n_cols, figsize=(14, 4 * n_rows))
axes = axes.flatten()  # Aplanar la matriz de ejes

for i, columna in enumerate(columnas):
    sizes = df_alg1[columna].value_counts(normalize=True) * 100
    if any(sizes > 80):
        columnas_mayor_80.append(columna)

    # Crear gráfico de dona para cada columna
    sizes = df_alg1[columna].value_counts()
    labels = sizes.index
    counts = sizes.values

    # Crear el gráfico de dona
    wedges, texts, autotexts = axes[i].pie(sizes, autopct='%1.1f%%', startangle=140, pctdistance=0.85)
    centre_circle = plt.Circle((0, 0), 0.70, fc='white')
    axes[i].add_artist(centre_circle)

    # Crear leyenda con conteos
    legend_labels = [f'{label} (Conteo: {count})' for label, count in zip(labels, counts)]
    axes[i].legend(wedges, legend_labels, title="Categorías", loc="upper left", bbox_to_anchor=(1, 1), fontsize='small')

    axes[i].set_title(f'Diagrama de dona - {columna}')

# Eliminar ejes vacíos si hay menos gráficos que subplots
for ax in axes[n:]:
    ax.remove()

plt.tight_layout(rect=[0, 0, 0.75, 1])  # Ajustar el espacio para la leyenda
plt.show()

In [ ]:
print("Columnas con categorías mayores al 80%:")
print(columnas_mayor_80)

In [ ]:
df_alg1 = df_alg1.drop(columns = columnas_mayor_80, axis=1)

#### Analisis Bivariado

##### Variables numéricas

###### Correlación y multicolinealidad

Se revisa la correlación de las variables númericas con la variable a predecir

In [ ]:
corr_matrix_num = df_alg1.select_dtypes(include=np.number).corr()
print("Correlaciones de las variables numericas con 'nota_03':")
print(corr_matrix_num["nota_03"].sort_values(ascending=False))

In [ ]:
threshold = 0.8 #este valor se utiliza para correlacion del 80 en adelante
filter_= np.abs(corr_matrix_num["nota_03"])> threshold
corr_features = corr_matrix_num.columns[filter_].tolist()
plt.figure(figsize=(12,8))
sns.heatmap(df_alg1[corr_features].corr(),annot=True, cmap = 'Blues', fmt=".2f")
plt.title("Correlacion entre variables numericas (Threshold: {})".format(threshold))
plt.show()

In [ ]:
# Variables numéricas
sns.pairplot(df_alg1[corr_features], diag_kind="kde")

Por la alta correlación de nota_01 y nota_02 con la variable respuesta, se realiza la prueba de correlación entre ellas para revisar multicolinealidad.

In [ ]:
correlacion_nota_01_02 = df_alg1["nota_01"].corr(df_alg1["nota_02"])
print("La correlación entre nota_01 y nota_02 es:", correlacion_nota_01_02)

Al encontrar multicolinealidad entre ellas, se decide eliminar nota_01, pues es la que menor correlación tiene con la variable respuesta..

In [ ]:
df_alg1 = df_alg1.drop(['nota_01'], axis=1)

##### Variables categóricas con más de dos categorías

Cramér's V usa la tabla de contingencia para calcular el estadístico Chi-cuadrado, que luego se convierte en una medida de la fuerza de la asociación entre las variables categóricas. El estadístico de pruba V va de 0 (sin asociación) a 1 (asociación perfecta).

In [ ]:
def cramers_v(x, y):
    # Crear una tabla de contingencia
    contingency_table = pd.crosstab(x, y)
    # Calcular el estadístico Chi-Cuadrado
    chi2_stat, _, _, _ = chi2_contingency(contingency_table)
    # Obtener el número de observaciones
    n = contingency_table.sum().sum()
    # Número de filas y columnas
    k = min(contingency_table.shape) - 1
    # Calcular Cramer's V
    return np.sqrt(chi2_stat / (n * k))

####### Correlaciones con matriz de Cramér's V

Se revisa la correlación de estas variables categóricas entre sí.


In [ ]:
# Lista de variables categóricas
categorical_vars = [col for col in df_alg1.select_dtypes(include=['object']).columns
                                   if len(df_alg1[col].unique()) > 2]

# Crear un DataFrame para almacenar los resultados
cramers_v_results = pd.DataFrame(index=categorical_vars, columns=categorical_vars)

# Calcular Cramér's V para cada par de variables categóricas
for var1 in categorical_vars:
    for var2 in categorical_vars:
        if var1 != var2:
            cramers_v_value = cramers_v(df_alg1[var1], df_alg1[var2])
            cramers_v_results.loc[var1, var2] = cramers_v_value
        else:
            # Para la diagonal (mismo par de variables), podemos poner NaN o 0
            cramers_v_results.loc[var1, var2] = np.nan

print("Matriz de Cramér's V entre variables categóricas:")
print(cramers_v_results)

En el caso de las variables categóricas ordinales, no encontramos problemas de colinealidad o multicolinealidad, considerando correlacines entre ellas con un nivel de significancia por encima del 80%.

##### Variables categóricas booleanas

####### Correlaciones con matriz de Cramér's V

Se revisa la correlación de estas variables categóricas entre sí.
Cramér's V usa la tabla de contingencia para calcular el estadístico Chi-cuadrado, que luego se convierte en una medida de la fuerza de la asociación entre las variables categóricas. El estadístico de pruba V va de 0 (sin asociación) a 1 (asociación perfecta).

In [ ]:
# Lista de variables categóricas
categorical_vars = [col for col in df_alg1.select_dtypes(include=['object']).columns
                                   if len(df_alg1[col].unique()) <= 2]

# Crear un DataFrame para almacenar los resultados
cramers_v_results = pd.DataFrame(index=categorical_vars, columns=categorical_vars)

# Calcular Cramér's V para cada par de variables categóricas
for var1 in categorical_vars:
    for var2 in categorical_vars:
        if var1 != var2:
            cramers_v_value = cramers_v(df_alg1[var1], df_alg1[var2])
            cramers_v_results.loc[var1, var2] = cramers_v_value
        else:
            # Para la diagonal (mismo par de variables), podemos poner NaN o 0
            cramers_v_results.loc[var1, var2] = np.nan

print("Matriz de Cramér's V entre variables categóricas:")
print(cramers_v_results)

En el caso de las variables categóricas booleanas, no encontramos problemas de colinealidad o multicolinealidad, considerando correlacines entre ellas con un nivel de significancia por encima del 80%.

#### Transformaciones

In [ ]:
# Se revisa el tipo de cada variable y se convierten en el que se considera acorde
df_alg1.info()

In [ ]:
# Convertir columnas enteras a int
columnas_enteras = ['nota_03']
df_alg1[columnas_enteras] = df_alg1[columnas_enteras].astype('int64')

In [ ]:
# Luego se valida que variables categóricas no pueden ser dummies por tener mas de dos valores únicos
columnas_categoricas_no_dummies = [col for col in df_alg1.select_dtypes(include=['object']).columns
                                   if len(df_alg1[col].unique()) > 2]
print("Columnas con más de dos valores únicos:")
print(columnas_categoricas_no_dummies)

In [ ]:
# Conversión de variables categóricas con dos valores únicos en variables dummies
df_alg1 = pd.get_dummies(df_alg1,
                         columns=[col for col in df_alg1.select_dtypes(include=['object']).columns
                                  if col not in columnas_categoricas_no_dummies],
                        drop_first=True)
df_alg1.info()

In [ ]:
# Para las variables categóricas con más de dos valores únicos se trabaja el método One-Hot para
# transformarlas en un conjunto de columnas binarias que indiquen la presencia o ausencia de la categoría

columnas_originales = df_alg1.columns.tolist()
df_alg1 = pd.get_dummies(df_alg1, columns=columnas_categoricas_no_dummies)
columnas_resultantes = df_alg1.columns.tolist()

# Obtener las nuevas columnas generadas por pd.get_dummies() para mas adelante
nuevas_columnas = list(set(columnas_resultantes) - set(columnas_originales))
print(nuevas_columnas)

In [ ]:
# Se normalizan las columnas númericas dividiendo cada valor por el máximo valor en la columna respectiva,
# de forma que los valores queden en un rango entre 0 y 1, exceptuando la variable a precedir 'nota_03'.
numericas_columns = []

for col in df_alg1.select_dtypes(include=np.number).columns:
    numericas_columns.append(col)
    if col != 'nota_03':
        df_alg1[col] = df_alg1[col] / df_alg1[col].max()

df_alg1[numericas_columns].head()

In [ ]:
df_alg1.info()

#### Análisis univariado VARIABLES NUEVAS

###### **Variables boolenas**

Para las variables booleanas hemos definido un umbral de 80-20. Donde las varibles que se distribuyan mayor de 80 y menor de 20, no serán significativas y se eliminarán del modelo.

In [ ]:
# Lista de columnas para graficar
nuevas_columnas.sort()
columnas = nuevas_columnas

columnas_mayor_80 = []

# Evaluar cada columna y crear gráficos
n = len(columnas)
n_cols = 2
n_rows = (n + 1) // n_cols  # Calcula el número de filas necesarias

fig, axes = plt.subplots(n_rows, n_cols, figsize=(14, 4 * n_rows))
axes = axes.flatten()  # Aplanar la matriz de ejes

for i, columna in enumerate(columnas):
    sizes = df_alg1[columna].value_counts(normalize=True) * 100
    if any(sizes > 80):
        columnas_mayor_80.append(columna)

    # Crear gráfico de dona para cada columna
    sizes = df_alg1[columna].value_counts()
    labels = sizes.index
    counts = sizes.values

    # Crear el gráfico de dona
    wedges, texts, autotexts = axes[i].pie(sizes, autopct='%1.1f%%', startangle=140, pctdistance=0.85)
    centre_circle = plt.Circle((0, 0), 0.70, fc='white')
    axes[i].add_artist(centre_circle)

    # Crear leyenda con conteos
    legend_labels = [f'{label} (Conteo: {count})' for label, count in zip(labels, counts)]
    axes[i].legend(wedges, legend_labels, title="Categorías", loc="upper left", bbox_to_anchor=(1, 1), fontsize='small')

    axes[i].set_title(f'Diagrama de dona - {columna}')

# Eliminar ejes vacíos si hay menos gráficos que subplots
for ax in axes[n:]:
    ax.remove()

plt.tight_layout(rect=[0, 0, 0.75, 1])  # Ajustar el espacio para la leyenda
plt.show()

In [ ]:
print("Columnas con categorías mayores al 80%:")
columnas_mayor_80.sort()
print(columnas_mayor_80)

In [ ]:
df_alg1 = df_alg1.drop(columns = ['razon_otro'], axis=1)

##### **Ultimas transformaciónes**

Al analizar las nuevas columnas resultantes de la dummización de las variables categóricas con más de dos valores únicos, identificamos que algunas categorías presentaban una baja representación, lo que podría influir negativamente en la capacidad predictiva del modelo. Para mitigar este efecto, decidimos agrupar valores con características similares dentro de estas variables categóricas. Este agrupamiento nos permitió mejorar la proporción de los datos en cada categoría, generando clases más equilibradas y con mayor representatividad. Además, renombramos estas nuevas variables agrupadas.

In [ ]:
df_alg2 = df_alg1.copy()

In [ ]:
# Incluimos la opción en_casa en otros para la variable madre_trab
df_alg2['madre_trab_otro'] = df_alg2['madre_trab_otro'] | ~df_alg2['madre_trab_otro'] & df_alg2['madre_trab_en_casa']
df_alg2 = df_alg2.drop('madre_trab_en_casa', axis = 1)

In [ ]:
# Incluimos la opción salud y profesor en servicios para la variable madre_trab, entendiendo servicios como trabajos
# del sector publico
df_alg2['madre_trab_servicios'] = (df_alg2['madre_trab_servicios'] | ~df_alg2['madre_trab_servicios'] & df_alg2['madre_trab_profesor']) | ~df_alg2['madre_trab_servicios'] & ~df_alg2['madre_trab_profesor'] & df_alg2['madre_trab_salud']
df_alg2 = df_alg2.drop(columns = ['madre_trab_profesor', 'madre_trab_salud'], axis = 1)

In [ ]:
# Incluimos la opción en_casa en otros para la variable padre_trab
df_alg2['padre_trab_otro'] = df_alg2['padre_trab_otro'] | ~df_alg2['padre_trab_otro'] & df_alg2['padre_trab_en_casa']
df_alg2 = df_alg2.drop('padre_trab_en_casa', axis = 1)

In [ ]:
# Incluimos la opción salud y profesor en servicios para la variable padre_trab, entendiendo servicios como trabajos
# del sector publico
df_alg2['padre_trab_servicios'] = (df_alg2['padre_trab_servicios'] | ~df_alg2['padre_trab_servicios'] & df_alg2['padre_trab_profesor']) | ~df_alg2['padre_trab_servicios'] & ~df_alg2['padre_trab_profesor'] & df_alg2['padre_trab_salud']
df_alg2 = df_alg2.drop(columns = ['padre_trab_profesor', 'padre_trab_salud'], axis = 1)

In [ ]:
columnas_resultantes = df_alg2.columns.tolist()

# Obtener las nuevas columnas generadas por pd.get_dummies() para mas adelante
nuevas_columnas = list(set(columnas_resultantes) - set(columnas_originales))

# Lista de columnas para graficar
nuevas_columnas.sort()
columnas = nuevas_columnas

columnas_mayor_80 = []

# Evaluar cada columna y crear gráficos
n = len(columnas)
n_cols = 2
n_rows = (n + 1) // n_cols  # Calcula el número de filas necesarias

fig, axes = plt.subplots(n_rows, n_cols, figsize=(14, 4 * n_rows))
axes = axes.flatten()  # Aplanar la matriz de ejes

for i, columna in enumerate(columnas):
    sizes = df_alg2[columna].value_counts(normalize=True) * 100
    if any(sizes > 80):
        columnas_mayor_80.append(columna)

    # Crear gráfico de dona para cada columna
    sizes = df_alg2[columna].value_counts()
    labels = sizes.index
    counts = sizes.values

    # Crear el gráfico de dona
    wedges, texts, autotexts = axes[i].pie(sizes, autopct='%1.1f%%', startangle=140, pctdistance=0.85)
    centre_circle = plt.Circle((0, 0), 0.70, fc='white')
    axes[i].add_artist(centre_circle)

    # Crear leyenda con conteos
    legend_labels = [f'{label} (Conteo: {count})' for label, count in zip(labels, counts)]
    axes[i].legend(wedges, legend_labels, title="Categorías", loc="upper left", bbox_to_anchor=(1, 1), fontsize='small')

    axes[i].set_title(f'Diagrama de dona - {columna}')

# Eliminar ejes vacíos si hay menos gráficos que subplots
for ax in axes[n:]:
    ax.remove()

plt.tight_layout(rect=[0, 0, 0.75, 1])  # Ajustar el espacio para la leyenda
plt.show()

Para las variables booleanas hemos definido un umbral de 80-20. Donde las varibles que se distribuyan mayor de 80 y menor de 20, no serán significativas y se eliminarán del modelo.

In [ ]:
print("Columnas con categorías mayores al 80%:")
columnas_mayor_80.sort()
print(columnas_mayor_80)

In [ ]:
df_alg2 = df_alg2.drop(columns = columnas_mayor_80, axis=1)

#### Modelos de regresión

##### Regresión logistica

In [ ]:
df_alg1["nota_03"].unique()

In [ ]:
def Transf(nota):
    if nota < 12:
        return False
    else:
        return True

In [ ]:
df_alg1['nota_03'] = df_alg1['nota_03'].apply(Transf)

In [ ]:
df_alg1['nota_03'].head()

In [ ]:
X_alg = df_alg1.drop("nota_03", axis=1)
y_alg = df_alg1["nota_03"]

# Separar los datos en conjuntos de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(X_alg, y_alg, test_size=0.3, random_state=113)

model = LogisticRegression() # definir el modelo
model.fit(x_train, y_train) # entrenar el modelo

y_pred_train = model.predict(x_train) # guardar la predicción para train
y_pred_test = model.predict(x_test) # guardar la predicción para test

In [ ]:
# Matriz de confusión:
cm = confusion_matrix(y_test, y_pred_test, labels=model.classes_) # guardar las clases para la matriz de confusión
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=model.classes_)
disp.plot();
print(cm)

In [ ]:
TP=cm[1,1] # El modelo dijo que eran 1 y en la realidad son 1, verdaderos positivos - 89
FP=cm[0,1] # El modelo dijo que eran 1, en la realidad son 0, falsos positivos - 19
FN=cm[1,0] # El modelo dijo que eran 0, en la realidad son 1, falsos negativos - 6
TN=cm[0,0] # El modelo dijo que eran 0 y en la realidad son 0, verdaderos negativos - 81

print(f"Accuracy test: {accuracy_score(y_test, y_pred_test)}")
print(f'Precicion: {TP/(TP+FP)}')
print(f'Recall (Sensibilidad)): {TP/(TP+FN)}')
print(f'F1-score:', f1_score(y_test, y_pred_test, average='binary'))
print(f'Especificidad: {TN/(FP+TN)}')

In [ ]:
# Metricas de entrenamiento
cm = confusion_matrix(y_train, y_pred_train, labels=model.classes_) # guardar las clases para la matriz de confusión

TP=cm[1,1] # El modelo dijo que eran 1 y en la realidad son 1, verdaderos positivos - 89
FP=cm[0,1] # El modelo dijo que eran 1, en la realidad son 0, falsos positivos - 19
FN=cm[1,0] # El modelo dijo que eran 0, en la realidad son 1, falsos negativos - 6
TN=cm[0,0] # El modelo dijo que eran 0 y en la realidad son 0, verdaderos negativos - 91

print(f"Accuracy train: {accuracy_score(y_train, y_pred_train)}")
print(f'Precicion: {TP/(TP+FP)}')
print(f'Recall (Sensibilidad)): {TP/(TP+FN)}')
print(f'F1-score:', f1_score(y_train, y_pred_train, average='binary'))
print(f'Especificidad: {TN/(FP+TN)}')

In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score
y_pred = model.predict_proba(x_test)[::,1]
fpr, tpr,_ =roc_curve(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)
plt.plot(fpr, tpr,marker='.',label='Logistic (auc= %0.3f)'%auc)
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.legend()
plt.show()

# **SEGUNDA ENTREGA**

## CÁLCULO DIFERENCIAL

### Selección de variables métodos de filtrado

#### Selección univariante


>Para realizar una mejor comparación de los métodos de seleccion de variables, aplicaremos los métodos *f-regression* y *mutual info regression* a la base de datos df_cal, la aplicaremos a 3 versiones de esta. La base original sin haberle hecho un tratamiento, la primera versión de nuestro tratmiento sólo con las trasformaciones donde hay 75 columnas o vairables y otra version despues de hacerle transformaciones y adicional agrupacciones que contiene 51, agrupaciones finales a criterio nuestro.
Esto con el objetivo de revisar si las predicciones mejoran al combinar trabajo manual con los métdos de selección o si por su parte estos trabajan mejor solos.

In [ ]:
print(len(df_cal.columns))

In [ ]:
print(len(df_cal1.columns))

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression,  f_classif, mutual_info_classif, chi2

In [ ]:
# Función de filtro de caracteristicas - stadis. scores
def select_kbest(X,y,score_f,k): #se establece una funcion que permite sacar varias funciones de evaluacion
    sel_kb = SelectKBest(score_func=score_f, k=k)
    sel_kb.fit(X,y)
    scores = sel_kb.scores_
    pvalues = sel_kb.pvalues_
    new_cols = sel_kb.get_support()
    print("Scores:\n", scores, "\nP-values:\n", pvalues)
    return new_cols, scores, pvalues

> Los siguientes metodos se usan en modelos de regresión

##### Método f regression



> Calcula el valor F y los p-valores asociados para evaluar la significancia de cada característica en relación con la variable objetivo, ayudando a seleccionar las características más relevantes.



###### Df_cal (Data frame con 75 columnas)

In [ ]:
df_cal.columns

In [ ]:
x = df_cal
y = df_cal['nota_03']
x_new, scores, pvalues = select_kbest(df_cal, y, f_regression, 25) # Obtener columnas seleccionadas
df_cal2 = df_cal.iloc[:,x_new] # Nuevo conjunto de datos
df_cal2.head() #se escogen los valores p mas pequeños, y las variables tiene relacion con la variable objetivo

>Estas son las 25 columnas que el modelo selecciona para trabajar en las predicciones.

In [ ]:
df_cal2.columns

In [ ]:
df_cal2.shape

In [ ]:
X_cal = df_cal2
y_cal = y
# Tranformación logarítmica
y_log = np.log(y_cal + 1)
# Separar los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_cal, y_log, test_size=0.3, random_state=59)
# Inicializar y entrenar el modelo
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train) # .fit: entrenar el modelo
# Realizar predicciones
lr_pred = lr_reg.predict(X_test)

In [ ]:
DF = 'df_cal, 75var'
Modelo = 'Test_f_regression'
# Evaluación del modelo (por ejemplo, precisión)
accuracy = lr_reg.score(X_test, y_test)
print("Accuracy:", accuracy)
# Metricas TEST
MSE = mean_squared_error(y_test, lr_pred)
print("MSE: %.2f" % MSE)
RMSE = math.sqrt(mean_squared_error(y_test, lr_pred))
print("RMSE %.2f" % RMSE)
MAE = mean_absolute_error(y_test, lr_pred)
print("MAE: %.2f" % MAE)
MAPE = mean_absolute_percentage_error(y_test, lr_pred)
print("MAPE: %.2f" % MAPE)
r2 = r2_score(y_test, lr_pred)
print("R2: %.2f" % r2_score(y_test, lr_pred))
# R2 ajustado
adj_r2 = 1 - (1-r2)*(len(y_test )-1)/(len(y_test)-X_test.shape[1]-1)
print("R2-adjusted: %.2f" % adj_r2)

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [accuracy],
    'MSE': [MSE],
    'RMSE': [RMSE],
    'MAE': [MAE],
    'MAPE': [MAPE],
    'R2': [r2],
    'R2_adjusted': [adj_r2]
})

# Concatenar con el DataFrame original
df_resultados = pd.concat([df_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(df_resultados)

In [ ]:
lr_pred = lr_reg.predict(X_train)

In [ ]:
DF = 'df_cal, 75var'
Modelo = 'Train_f_regression'
# Metricas
accuracy = lr_reg.score(X_train, y_train)
print("Accuracy: %.2f" % accuracy)
MSE = mean_squared_error(y_train, lr_pred)
print("MSE: %.2f" % MSE)
RMSE = math.sqrt(mean_squared_error(y_train, lr_pred))
print("RMSE %.2f" % RMSE)
MAE = mean_absolute_error(y_train, lr_pred)
print("MAE: %.2f" % MAE)
MAPE = mean_absolute_percentage_error(y_train, lr_pred)
print("MAPE: %.2f" % MAPE)
r2 = r2_score(y_train, lr_pred)
print("R2: %.2f" % r2)
# R2 ajustado
adj_r2 = 1 - (1-r2)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
print("R2-adjusted: %.2f" % adj_r2)

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [accuracy],
    'MSE': [MSE],
    'RMSE': [RMSE],
    'MAE': [MAE],
    'MAPE': [MAPE],
    'R2': [r2],
    'R2_adjusted': [adj_r2]
})

# Concatenar con el DataFrame original
df_resultados = pd.concat([df_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(df_resultados)

###### Df_cal1 (Data frame con 51 columnas)

In [ ]:
df_cal1.columns

In [ ]:
x = df_cal1
y = df_cal1['nota_03']
x_new, scores, pvalues = select_kbest(df_cal1, y, f_regression, 25) # Obtener columnas seleccionadas
df_cal3 = df_cal1.iloc[:,x_new] # Nuevo conjunto de datos
df_cal3.head() #se escogen los valores p mas pequeños, y las variables tiene relacion con la variable objetivo

>Estas son las 25 columnas que el modelo selecciona para trabajar en las predicciones.

In [ ]:
df_cal3.columns

In [ ]:
df_cal3.shape

In [ ]:
X_cal = df_cal3
y_cal = y
# Tranformación logarítmica
y_log = np.log(y_cal + 1)
# Separar los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_cal, y_log, test_size=0.3, random_state=59)
# Inicializar y entrenar el modelo
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train) # .fit: entrenar el modelo
# Realizar predicciones
lr_pred = lr_reg.predict(X_test)

In [ ]:
DF = 'df_cal, 51var'
Modelo = 'Test_f_regression'
# Evaluación del modelo (por ejemplo, precisión)
accuracy = lr_reg.score(X_test, y_test)
print("Accuracy:", accuracy)
# Metricas TEST
MSE = mean_squared_error(y_test, lr_pred)
print("MSE: %.2f" % MSE)
RMSE = math.sqrt(mean_squared_error(y_test, lr_pred))
print("RMSE %.2f" % RMSE)
MAE = mean_absolute_error(y_test, lr_pred)
print("MAE: %.2f" % MAE)
MAPE = mean_absolute_percentage_error(y_test, lr_pred)
print("MAPE: %.2f" % MAPE)
r2 = r2_score(y_test, lr_pred)
print("R2: %.2f" % r2_score(y_test, lr_pred))
# R2 ajustado
adj_r2 = 1 - (1-r2)*(len(y_test )-1)/(len(y_test)-X_test.shape[1]-1)
print("R2-adjusted: %.2f" % adj_r2)

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [accuracy],
    'MSE': [MSE],
    'RMSE': [RMSE],
    'MAE': [MAE],
    'MAPE': [MAPE],
    'R2': [r2],
    'R2_adjusted': [adj_r2]
})

# Concatenar con el DataFrame original
df_resultados = pd.concat([df_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(df_resultados)

In [ ]:
lr_pred = lr_reg.predict(X_train)

In [ ]:
DF = 'df_cal, 51var'
Modelo = 'Train_f_regression'
# Metricas
accuracy = lr_reg.score(X_train, y_train)
print("Accuracy: %.2f" % accuracy)
MSE = mean_squared_error(y_train, lr_pred)
print("MSE: %.2f" % MSE)
RMSE = math.sqrt(mean_squared_error(y_train, lr_pred))
print("RMSE %.2f" % RMSE)
MAE = mean_absolute_error(y_train, lr_pred)
print("MAE: %.2f" % MAE)
MAPE = mean_absolute_percentage_error(y_train, lr_pred)
print("MAPE: %.2f" % MAPE)
r2 = r2_score(y_train, lr_pred)
print("R2: %.2f" % r2)
# R2 ajustado
adj_r2 = 1 - (1-r2)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
print("R2-adjusted: %.2f" % adj_r2)

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [accuracy],
    'MSE': [MSE],
    'RMSE': [RMSE],
    'MAE': [MAE],
    'MAPE': [MAPE],
    'R2': [r2],
    'R2_adjusted': [adj_r2]
})

# Concatenar con el DataFrame original
df_resultados = pd.concat([df_resultados, nueva_fila], ignore_index=True)

In [ ]:
# Mostrar el DataFrame con los resultados
df_resultados

>El modelo de selección *f-regression* muestra un desmejoramiento (aunque no significativo) de las métricas de evaluación del modelo en su etapa de entrenamiento. Adicionalmente se aprecia que este método arroja un r2 ajustado un poco mejores, para la base de datos con las transformaciones y agrupaciones propias, es decir que funciona un poco mejor al combinar el metodo y nuestra manipulación.

##### Método mutual info regression

> Este método estima cuánta información sobre la variable objetivo se puede obtener de cada característica, siendo útil para la selección de características

###### Df_cal (Data frame con 75 columnas)

In [ ]:
y = df_cal['nota_03']
x_new, scores, pvalues = select_kbest(df_cal, y, mutual_info_regression, 31) # Obtener columnas seleciconadas
df_cal4 = df_cal.iloc[:,x_new] # Nuevo conjunto de datos
df_cal4.head() #se escogen los valores p mas pequeños, y las variables tiene relacion con la variable objetivo

In [ ]:
X_cal = df_cal4.drop('nota_03', axis=1)
y_cal = df_cal["nota_03"]
# Tranformación logarítmica
y_log = np.log(y_cal + 1)
# Separar los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_cal, y_log, test_size=0.3, random_state=59)
# Inicializar y entrenar el modelo GBM
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train) # .fit: entrenar el modelo
# Realizar predicciones
lr_pred = lr_reg.predict(X_test)

In [ ]:
DF = 'df_cal, 75var'
Modelo = 'Test_mutal_info'
# Evaluación del modelo (por ejemplo, precisión)
accuracy = lr_reg.score(X_test, y_test)
print("Accuracy:", accuracy)
# Metricas TEST
MSE = mean_squared_error(y_test, lr_pred)
print("MSE: %.2f" % MSE)
RMSE = math.sqrt(mean_squared_error(y_test, lr_pred))
print("RMSE %.2f" % RMSE)
MAE = mean_absolute_error(y_test, lr_pred)
print("MAE: %.2f" % MAE)
MAPE = mean_absolute_percentage_error(y_test, lr_pred)
print("MAPE: %.2f" % MAPE)
r2 = r2_score(y_test, lr_pred)
print("R2: %.2f" % r2_score(y_test, lr_pred))
# R2 ajustado
adj_r2 = 1 - (1-r2)*(len(y_test )-1)/(len(y_test)-X_test.shape[1]-1)
print("R2-adjusted: %.2f" % adj_r2)

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'Modelo': [Modelo],
    'DF': [DF],
    'Accuracy': [accuracy],
    'MSE': [MSE],
    'RMSE': [RMSE],
    'MAE': [MAE],
    'MAPE': [MAPE],
    'R2': [r2],
    'R2_adjusted': [adj_r2]
})

# Concatenar con el DataFrame original
df_resultados = pd.concat([df_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(df_resultados)

In [ ]:
lr_pred = lr_reg.predict(X_train)

In [ ]:
DF = 'df_cal, 75var'
Modelo = 'Train_mutal_info'
# Metricas
accuracy = lr_reg.score(X_train, y_train)
print("Accuracy: %.2f" % accuracy)
MSE = mean_squared_error(y_train, lr_pred)
print("MSE: %.2f" % MSE)
RMSE = math.sqrt(mean_squared_error(y_train, lr_pred))
print("RMSE %.2f" % RMSE)
MAE = mean_absolute_error(y_train, lr_pred)
print("MAE: %.2f" % MAE)
MAPE = mean_absolute_percentage_error(y_train, lr_pred)
print("MAPE: %.2f" % MAPE)
r2 = r2_score(y_train, lr_pred)
print("R2: %.2f" % r2)
# R2 ajustado
adj_r2 = 1 - (1-r2)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
print("R2-adjusted: %.2f" % adj_r2)

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [accuracy],
    'MSE': [MSE],
    'RMSE': [RMSE],
    'MAE': [MAE],
    'MAPE': [MAPE],
    'R2': [r2],
    'R2_adjusted': [adj_r2]
})

# Concatenar con el DataFrame original
df_resultados = pd.concat([df_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(df_resultados)

###### Df_cal1 (Data frame con 51 columnas)

In [ ]:
y = df_cal['nota_03']
x_new, scores, pvalues = select_kbest(df_cal, y, mutual_info_regression, 31) # Obtener columnas seleciconadas
df_cal5 = df_cal.iloc[:,x_new] # Nuevo conjunto de datos
df_cal5.head() #se escogen los valores p mas pequeños, y las variables tiene relacion con la variable objetivo

In [ ]:
X_cal = df_cal5.drop('nota_03', axis=1)
y_cal = df_cal["nota_03"]
# Tranformación logarítmica
y_log = np.log(y_cal + 1)
# Separar los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_cal, y_log, test_size=0.3, random_state=59)
# Inicializar y entrenar el modelo GBM
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train) # .fit: entrenar el modelo
# Realizar predicciones
lr_pred = lr_reg.predict(X_test)

In [ ]:
DF = 'df_cal, 51var'
Modelo = 'Test_mutal_info'
# Evaluación del modelo (por ejemplo, precisión)
accuracy = lr_reg.score(X_test, y_test)
print("Accuracy:", accuracy)
# Metricas TEST
MSE = mean_squared_error(y_test, lr_pred)
print("MSE: %.2f" % MSE)
RMSE = math.sqrt(mean_squared_error(y_test, lr_pred))
print("RMSE %.2f" % RMSE)
MAE = mean_absolute_error(y_test, lr_pred)
print("MAE: %.2f" % MAE)
MAPE = mean_absolute_percentage_error(y_test, lr_pred)
print("MAPE: %.2f" % MAPE)
r2 = r2_score(y_test, lr_pred)
print("R2: %.2f" % r2_score(y_test, lr_pred))
# R2 ajustado
adj_r2 = 1 - (1-r2)*(len(y_test )-1)/(len(y_test)-X_test.shape[1]-1)
print("R2-adjusted: %.2f" % adj_r2)

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [accuracy],
    'MSE': [MSE],
    'RMSE': [RMSE],
    'MAE': [MAE],
    'MAPE': [MAPE],
    'R2': [r2],
    'R2_adjusted': [adj_r2]
})

# Concatenar con el DataFrame original
df_resultados = pd.concat([df_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(df_resultados)

In [ ]:
lr_pred = lr_reg.predict(X_train)

In [ ]:
DF = 'df_cal, 51var'
Modelo = 'Train_mutal_info'
# Metricas
accuracy = lr_reg.score(X_train, y_train)
print("Accuracy: %.2f" % accuracy)
MSE = mean_squared_error(y_train, lr_pred)
print("MSE: %.2f" % MSE)
RMSE = math.sqrt(mean_squared_error(y_train, lr_pred))
print("RMSE %.2f" % RMSE)
MAE = mean_absolute_error(y_train, lr_pred)
print("MAE: %.2f" % MAE)
MAPE = mean_absolute_percentage_error(y_train, lr_pred)
print("MAPE: %.2f" % MAPE)
r2 = r2_score(y_train, lr_pred)
print("R2: %.2f" % r2)
# R2 ajustado
adj_r2 = 1 - (1-r2)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
print("R2-adjusted: %.2f" % adj_r2)

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [accuracy],
    'MSE': [MSE],
    'RMSE': [RMSE],
    'MAE': [MAE],
    'MAPE': [MAPE],
    'R2': [r2],
    'R2_adjusted': [adj_r2]
})

# Concatenar con el DataFrame original
df_resultados = pd.concat([df_resultados, nueva_fila], ignore_index=True)

In [ ]:
# Mostrar el DataFrame con los resultados
df_resultados

Revisando las diferencias en los resultados del r2 ajustado para train y test, el modelo que presenta un rendimiento más equilibrado y mejor capacidad de generalización es el df_cal de 51 variables con el método f_regression. Aunque hay un ligero descenso en el R² ajustado en el conjunto de prueba, no es alarmante y es la menor diferencia.

In [ ]:
df_resultados[(df_resultados['Modelo'] == 'Test_original') | (df_resultados['Modelo'] == 'Test_f_regression')
                | (df_resultados['Modelo'] == 'Test_mutal_info')]

>Las mejores métricas para el modelo de Test las presenta el DataFrame al que se le aplican todas las transformaciones y queda con 51 variables junto con el método f regression.

>Este modelo muestra el Accuracy más alto, los menores errores (aunque el MAPE es el segundo más bajo), y los valores más altos tanto para R² como para R² ajustado.

In [ ]:
df_resultados[(df_resultados['Modelo'] == 'Train_original') | (df_resultados['Modelo'] == 'Train_f_regression')
                | (df_resultados['Modelo'] == 'Train_mutal_info')]

>Las mejores métricas para el modelo de Train las presenta el DataFrame original, ya que, en comparación con el segundo mejor, que es el DataFrame con todas las transformaciones y el método mutual info regression, el modelo original obtiene mejores resultados.

>Esto se debe a que, de las cuatro métricas de error, presenta los menores valores en dos de ellas, mientras que las otras dos son iguales. Además, observamos que el DataFrame original tiene un valor más alto en cuanto a Accuracy y R². El valor del R² ajustado es mejor en el DataFrame con todas las transformaciones y el método mutual info regression, debido a que incluye más variables. Sin embargo, este último no logra superar el rendimiento general del DataFrame original.

In [ ]:
df_resultados[(df_resultados['DF'] == 'df_cal, orig')
              | ((df_resultados['DF'] == 'df_cal, 51var') & (df_resultados['Modelo'] == 'Test_mutal_info'))
              | ((df_resultados['DF'] == 'df_cal, 51var') & (df_resultados['Modelo'] == 'Train_mutal_info'))]

>De lo anterior concluimos que el mejor modelo debe ser el que toma el DF con todas las transformaciones y ademas le aplica el método *f regression* ya que presenta mejor metrica en Test y menor GAP entre Train y Test.

#### Umbral de varianza

###### Df_cal (Data frame con 75 columnas)

In [ ]:
#Se realiza una copia del data frame original pre-procesado
df_cal6= df_cal.copy()

In [ ]:
print(len(df_cal6.columns))

In [ ]:
y1 = df_cal6['nota_03']
df_cal6 = df_cal6.drop('nota_03', axis=1)

In [ ]:
# Separación de caracteristicas y objetivo, variables predictoras y objetivo
X_class = df_cal6
y_class = y1

print(X_class.shape)
print(y_class.shape)

In [ ]:
#Separacion test and train
from sklearn.model_selection import train_test_split

#Separacion de conjuntos de train (80%) y test(20%)
X_train, X_test, y_train, y_test = train_test_split(X_class,y_class, test_size= 0.3, random_state=42)

#Imprimir tamaño del dataset
print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)

In [ ]:
#seleccion de variables usando el metodo Variance Threshold, este metodo selecciona las caracteristicas cuya varianza supera un cierto umbral entonces,
#si la varianza de una variable es pequeña la probabilidad de que este contenga un valor igual o parecido es muy elevada por tanto, poco aportará al algoritmo
from sklearn.feature_selection import VarianceThreshold

#Función de filtro de caracteristicas, que recibe dos ibservaciones
def variance_threshold(X,th):
    #definimos el filtrador
    var_thres=VarianceThreshold(threshold=th)
    #Convertimos todas la columnas a numericas sino es posible las eliminamos
    X = X.apply(pd.to_numeric, errors='coerce').dropna(axis=1)
    #ajustamos el filtrador
    var_thres.fit(X)
    #retorna un arreglo con las variables que vamos a conservar
    new_cols = var_thres.get_support()
    #retornar los nombres de las columnas en lugar de la matriz booleana
    return X.columns[new_cols]

In [ ]:
# Obtener columnas seleccionadas, estableciendo un thershold de 20%
# el flitrador elimina las variables que tengan desde el 80% de observaciones iguales (como las constantes)
X_new_class = variance_threshold(X_class, 0.20)
# definir nuevo dataframe a partir, de X_class que es el df de las observaciones sin la var target, accediento a la columnas y filas predichas
df_classification_new = X_class[X_new_class]
df_classification_new.head()

In [ ]:
df_classification_new.shape

In [ ]:
X_cal = df_classification_new
#X_cal = df_selected
y_cal = y1

# Tranformación logarítmica
y_log = np.log(y_cal + 1)
# Separar los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_cal, y_log, test_size=0.3, random_state=59)
# Inicializar y entrenar el modelo GBM
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train) # .fit: entrenar el modelo
# Realizar predicciones
lr_pred = lr_reg.predict(X_test)

In [ ]:
DF = 'df_cal, 75var'
Modelo = 'Test_variance_threshold'
# Evaluación del modelo (por ejemplo, precisión)
accuracy = lr_reg.score(X_test, y_test)
print("Accuracy:", accuracy)
# Metricas TEST
MSE = mean_squared_error(y_test, lr_pred)
print("MSE: %.2f" % MSE)
RMSE = math.sqrt(mean_squared_error(y_test, lr_pred))
print("RMSE %.2f" % RMSE)
MAE = mean_absolute_error(y_test, lr_pred)
print("MAE: %.2f" % MAE)
MAPE = mean_absolute_percentage_error(y_test, lr_pred)
print("MAPE: %.2f" % MAPE)
r2 = r2_score(y_test, lr_pred)
print("R2: %.2f" % r2_score(y_test, lr_pred))
# R2 ajustado
adj_r2 = 1 - (1-r2)*(len(y_test )-1)/(len(y_test)-X_test.shape[1]-1)
print("R2-adjusted: %.2f" % adj_r2)

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [accuracy],
    'MSE': [MSE],
    'RMSE': [RMSE],
    'MAE': [MAE],
    'MAPE': [MAPE],
    'R2': [r2],
    'R2_adjusted': [adj_r2]
})

# Concatenar con el DataFrame original
df_resultados = pd.concat([df_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(df_resultados)

In [ ]:
lr_pred = lr_reg.predict(X_train)

In [ ]:
DF = 'df_cal, 75var'
Modelo = 'Train_variance_threshold'
# Metricas
accuracy = lr_reg.score(X_train, y_train)
print("Accuracy: %.2f" % accuracy)
MSE = mean_squared_error(y_train, lr_pred)
print("MSE: %.2f" % MSE)
RMSE = math.sqrt(mean_squared_error(y_train, lr_pred))
print("RMSE %.2f" % RMSE)
MAE = mean_absolute_error(y_train, lr_pred)
print("MAE: %.2f" % MAE)
MAPE = mean_absolute_percentage_error(y_train, lr_pred)
print("MAPE: %.2f" % MAPE)
r2 = r2_score(y_train, lr_pred)
print("R2: %.2f" % r2)
# R2 ajustado
adj_r2 = 1 - (1-r2)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
print("R2-adjusted: %.2f" % adj_r2)

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [accuracy],
    'MSE': [MSE],
    'RMSE': [RMSE],
    'MAE': [MAE],
    'MAPE': [MAPE],
    'R2': [r2],
    'R2_adjusted': [adj_r2]
})

# Concatenar con el DataFrame original
df_resultados = pd.concat([df_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(df_resultados)

###### Df_cal (Data frame con 51 columnas)

In [ ]:
#Se realiza una copia del data frame original pre-procesado
df_cal6= df_cal1.copy()

In [ ]:
print(len(df_cal6.columns))

In [ ]:
y1 = df_cal6['nota_03']
df_cal6 = df_cal6.drop('nota_03', axis=1)

In [ ]:
# Separación de caracteristicas y objetivo, variables predictoras y objetivo
X_class = df_cal6
y_class = y1

print(X_class.shape)
print(y_class.shape)

In [ ]:
#Separacion test and train
from sklearn.model_selection import train_test_split

#Separacion de conjuntos de train (80%) y test(20%)
X_train, X_test, y_train, y_test = train_test_split(X_class,y_class, test_size= 0.3, random_state=42)

#Imprimir tamaño del dataset
print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)

In [ ]:
# Obtener columnas seleccionadas, estableciendo un thershold de 20%
# el flitrador elimina las variables que tengan desde el 80% de observaciones iguales (como las constantes)
X_new_class = variance_threshold(X_class, 0.20)
# definir nuevo dataframe a partir, de X_class que es el df de las observaciones sin la var target, accediento a la columnas y filas predichas
df_classification_new = X_class[X_new_class]
df_classification_new.head()

In [ ]:
df_classification_new.shape

In [ ]:
X_cal = df_classification_new
#X_cal = df_selected
y_cal = y1

# Tranformación logarítmica
y_log = np.log(y_cal + 1)
# Separar los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_cal, y_log, test_size=0.3, random_state=59)
# Inicializar y entrenar el modelo GBM
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train) # .fit: entrenar el modelo
# Realizar predicciones
lr_pred = lr_reg.predict(X_test)

In [ ]:
DF = 'df_cal, 51var'
Modelo = 'Test_variance_threshold'
# Evaluación del modelo (por ejemplo, precisión)
accuracy = lr_reg.score(X_test, y_test)
print("Accuracy:", accuracy)
# Metricas TEST
MSE = mean_squared_error(y_test, lr_pred)
print("MSE: %.2f" % MSE)
RMSE = math.sqrt(mean_squared_error(y_test, lr_pred))
print("RMSE %.2f" % RMSE)
MAE = mean_absolute_error(y_test, lr_pred)
print("MAE: %.2f" % MAE)
MAPE = mean_absolute_percentage_error(y_test, lr_pred)
print("MAPE: %.2f" % MAPE)
r2 = r2_score(y_test, lr_pred)
print("R2: %.2f" % r2_score(y_test, lr_pred))
# R2 ajustado
adj_r2 = 1 - (1-r2)*(len(y_test )-1)/(len(y_test)-X_test.shape[1]-1)
print("R2-adjusted: %.2f" % adj_r2)

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [accuracy],
    'MSE': [MSE],
    'RMSE': [RMSE],
    'MAE': [MAE],
    'MAPE': [MAPE],
    'R2': [r2],
    'R2_adjusted': [adj_r2]
})

# Concatenar con el DataFrame original
df_resultados = pd.concat([df_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(df_resultados)

In [ ]:
lr_pred = lr_reg.predict(X_train)

In [ ]:
DF = 'df_cal, 51var'
Modelo = 'Train_variance_threshold'
# Metricas
accuracy = lr_reg.score(X_train, y_train)
print("Accuracy: %.2f" % accuracy)
MSE = mean_squared_error(y_train, lr_pred)
print("MSE: %.2f" % MSE)
RMSE = math.sqrt(mean_squared_error(y_train, lr_pred))
print("RMSE %.2f" % RMSE)
MAE = mean_absolute_error(y_train, lr_pred)
print("MAE: %.2f" % MAE)
MAPE = mean_absolute_percentage_error(y_train, lr_pred)
print("MAPE: %.2f" % MAPE)
r2 = r2_score(y_train, lr_pred)
print("R2: %.2f" % r2)
# R2 ajustado
adj_r2 = 1 - (1-r2)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
print("R2-adjusted: %.2f" % adj_r2)

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [accuracy],
    'MSE': [MSE],
    'RMSE': [RMSE],
    'MAE': [MAE],
    'MAPE': [MAPE],
    'R2': [r2],
    'R2_adjusted': [adj_r2]
})

# Concatenar con el DataFrame original
df_resultados = pd.concat([df_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(df_resultados)

El aumento en los errrores y la desmejoría de las métricas en comparación con la selección univariante es notoria. El método f regression sigue siendo el más óptimo.

### Optimización de hiper-parámetros

#### Regresión Ridge

In [ ]:
#Se realiza una copia del data frame original pre-procesado
df_cal7= df_cal.copy()
X = df_cal7.drop('nota_03', axis=1)
Y = df_cal7['nota_03']

In [ ]:
from sklearn.linear_model import Ridge, LogisticRegression, LinearRegression
from sklearn.model_selection import GridSearchCV, cross_val_score

RidgeRegression = Ridge()
hyperParameters = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40,45,50,55,100]}
ridgeRegressor = GridSearchCV(RidgeRegression, hyperParameters, scoring='neg_mean_squared_error')
ridgeRegressor.fit(X,y)

In [ ]:
print("Best value for lambda : ",ridgeRegressor.best_params_)
print("Best score for cost function: ", ridgeRegressor.best_score_)

#### Regresión Lasso

In [ ]:
from sklearn.linear_model import Lasso

LassoRegression = Lasso()
hyperParameters = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40,45,50,55,100]}
LassoRegressor = GridSearchCV(LassoRegression, hyperParameters, scoring='neg_mean_squared_error')
LassoRegressor.fit(X,y)

In [ ]:
print("Best value for lambda : ",LassoRegressor.best_params_)
print("Best score for cost function: ", LassoRegressor.best_score_)

#### Comparativa

In [ ]:
# Definir función
from sklearn.model_selection import train_test_split
def test(models, X, y, iterations = 100):
    results = {}
    for i in models:
        mse_train = []
        mse_test = []
        for j in range(iterations):
            X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                                test_size= 0.2)
            mse_test.append(metrics.mean_squared_error(y_test,
                                            models[i].fit(X_train,
                                                         y_train).predict(X_test)))
            mse_train.append(metrics.mean_squared_error(y_train,
                                             models[i].fit(X_train,
                                                          y_train).predict(X_train)))
        results[i] = [np.mean(mse_train), np.mean(mse_test)]
    return pd.DataFrame(results)

In [ ]:
models = {'OLS': LinearRegression(),
         'Lasso': Lasso(),
         'Ridge': Ridge(),}

In [ ]:
pd.options.display.float_format = '{:.5f}'.format
test(models, X, y)

In [ ]:
lasso_params = {'alpha':[0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03]}
ridge_params = {'alpha':[0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03]}

models2 = {'OLS': LinearRegression(),
           'Lasso': GridSearchCV(Lasso(),
                               param_grid=lasso_params).fit(X, y).best_estimator_,
           'Ridge': GridSearchCV(Ridge(),
                               param_grid=ridge_params).fit(X, y).best_estimator_,}

In [ ]:
test(models2, X, y)

### Otros modelos

#### Arboles de deción

In [ ]:
# Dividir en independientes y dependiente
X, y = df_cal3, df_cal1['nota_03']

# Dividir en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 123, test_size= 0.2)

# Definir modelo
regressor = DecisionTreeRegressor(max_depth=4)   # Cambiado a DecisionTreeRegressor

# Entrenar modelo
regressor.fit(X_train, y_train)

In [ ]:
DF = 'df_cal, 51var'
Modelo = 'Decision_tree_train'
y_pred = regressor.predict(X_train)
MSE = mean_squared_error(y_train, y_pred)
print("MSE: %.2f" % MSE)
RMSE = math.sqrt(mean_squared_error(y_train, y_pred))
print("RMSE %.2f" % RMSE)
MAE = mean_absolute_error(y_train, y_pred)
print("MAE: %.2f" % MAE)
MAPE = mean_absolute_percentage_error(y_train, y_pred)
print("MAPE: %.2f" % MAPE)
r2 = r2_score(y_train, y_pred)
print("R2: %.2f" % r2)
# R2 ajustado
adj_r2 = 1 - (1-r2)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
print("R2-adjusted: %.2f" % adj_r2)

In [ ]:
modelos_resultados = pd.DataFrame(columns=['DF', 'Modelo', 'Accuracy', 'Precisión', 'Recall', 'F1-score', 'Especificidad'])

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'MSE': [MSE],
    'RMSE': [RMSE],
    'MAE': [MAE],
    'MAPE': [MAPE],
    'R2': [r2],
    'R2_adjusted': [adj_r2]
})

# Concatenar con el DataFrame original
modelos_resultados = pd.concat([modelos_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(modelos_resultados)

In [ ]:
DF = 'df_cal, 51var'
Modelo = 'Decision_tree_test'
y_pred = regressor.predict(X_test)
# Metricas TEST
MSE = mean_squared_error(y_test, y_pred)
print("MSE: %.2f" % MSE)
RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE %.2f" % RMSE)
MAE = mean_absolute_error(y_test, y_pred)
print("MAE: %.2f" % MAE)
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE: %.2f" % MAPE)
r2 = r2_score(y_test, y_pred)
print("R2: %.2f" % r2_score(y_test, y_pred))
# R2 ajustado
adj_r2 = 1 - (1-r2)*(len(y_test )-1)/(len(y_test)-X_test.shape[1]-1)
print("R2-adjusted: %.2f" % adj_r2)

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'MSE': [MSE],
    'RMSE': [RMSE],
    'MAE': [MAE],
    'MAPE': [MAPE],
    'R2': [r2],
    'R2_adjusted': [adj_r2]
})

# Concatenar con el DataFrame original
modelos_resultados = pd.concat([modelos_resultados, nueva_fila], ignore_index=True)

In [ ]:
# Mostrar el DataFrame con los resultados
modelos_resultados

>El modelo de arbol de deción muestra unos peores resultado que el modelo de regresión lineal.

#### Bosques Aleatorios

In [ ]:
model = RandomForestRegressor(n_estimators = 100, # número de arboles
                               criterion = 'squared_error',  #Se refiere al error cuadrático medio.
                               max_depth = 4, # estrategia de parada
                               max_leaf_nodes = 10, # estrategia de parada
                               max_features = None,
                               oob_score = False,
                               n_jobs = -1,
                               random_state = 123)
model.fit(X_train, y_train)

In [ ]:
DF = 'df_cal, 51var'
Modelo = 'Random_forest_train'
y_pred = model.predict(X_train)
MSE = mean_squared_error(y_train, y_pred)
print("MSE: %.2f" % MSE)
RMSE = math.sqrt(mean_squared_error(y_train, y_pred))
print("RMSE %.2f" % RMSE)
MAE = mean_absolute_error(y_train, y_pred)
print("MAE: %.2f" % MAE)
MAPE = mean_absolute_percentage_error(y_train, y_pred)
print("MAPE: %.2f" % MAPE)
r2 = r2_score(y_train, y_pred)
print("R2: %.2f" % r2)
# R2 ajustado
adj_r2 = 1 - (1-r2)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
print("R2-adjusted: %.2f" % adj_r2)

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [Accuracy],
    'MSE': [MSE],
    'RMSE': [RMSE],
    'MAE': [MAE],
    'MAPE': [MAPE],
    'R2': [r2],
    'R2_adjusted': [adj_r2]
})

# Concatenar con el DataFrame original
modelos_resultados = pd.concat([modelos_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(modelos_resultados)

In [ ]:
DF = 'df_cal, 51var'
Modelo = 'Random_forest_test'
y_pred = model.predict(X_test)
MSE = mean_squared_error(y_test, y_pred)
print("MSE: %.2f" % MSE)
RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE %.2f" % RMSE)
MAE = mean_absolute_error(y_test, y_pred)
print("MAE: %.2f" % MAE)
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE: %.2f" % MAPE)
r2 = r2_score(y_test, y_pred)
print("R2: %.2f" % r2)
# R2 ajustado
adj_r2 = 1 - (1-r2)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
print("R2-adjusted: %.2f" % adj_r2)

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [Accuracy],
    'MSE': [MSE],
    'RMSE': [RMSE],
    'MAE': [MAE],
    'MAPE': [MAPE],
    'R2': [r2],
    'R2_adjusted': [adj_r2]
})

# Concatenar con el DataFrame original
modelos_resultados = pd.concat([modelos_resultados, nueva_fila], ignore_index=True)

In [ ]:
# Mostrar el DataFrame con los resultados
modelos_resultados

#### Gradient Boosting

In [ ]:
X, y = df_cal3, df_cal1['nota_03']

model = GradientBoostingRegressor()

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

model.fit(X_train, y_train)

In [ ]:
DF = 'df_cal, 51var'
Modelo = 'Gradient_Regressor_test'
y_pred = model.predict(X_test)
Accuracy = np.NaN
print("Accuracy:", Accuracy)
MSE = mean_squared_error(y_test, y_pred)
print("MSE: %.2f" % MSE)
RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE %.2f" % RMSE)
MAE = mean_absolute_error(y_test, y_pred)
print("MAE: %.2f" % MAE)
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE: %.2f" % MAPE)
r2 = r2_score(y_test, y_pred)
print("R2: %.2f" % r2)
# R2 ajustado
adj_r2 = 1 - (1-r2)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
print("R2-adjusted: %.2f" % adj_r2)

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [Accuracy],
    'MSE': [MSE],
    'RMSE': [RMSE],
    'MAE': [MAE],
    'MAPE': [MAPE],
    'R2': [r2],
    'R2_adjusted': [adj_r2]
})

# Concatenar con el DataFrame original
modelos_resultados = pd.concat([modelos_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(modelos_resultados)

In [ ]:
DF = 'df_cal, 51var'
Modelo = 'Gradient_Regressor_train'
y_pred = model.predict(X_train)
Accuracy = np.NaN
print("Accuracy:", Accuracy)
MSE = mean_squared_error(y_train, y_pred)
print("MSE: %.2f" % MSE)
RMSE = math.sqrt(mean_squared_error(y_train, y_pred))
print("RMSE %.2f" % RMSE)
MAE = mean_absolute_error(y_train, y_pred)
print("MAE: %.2f" % MAE)
MAPE = mean_absolute_percentage_error(y_train, y_pred)
print("MAPE: %.2f" % MAPE)
r2 = r2_score(y_train, y_pred)
print("R2: %.2f" % r2)
# R2 ajustado
adj_r2 = 1 - (1-r2)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
print("R2-adjusted: %.2f" % adj_r2)

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [Accuracy],
    'MSE': [MSE],
    'RMSE': [RMSE],
    'MAE': [MAE],
    'MAPE': [MAPE],
    'R2': [r2],
    'R2_adjusted': [adj_r2]
})

# Concatenar con el DataFrame original
modelos_resultados = pd.concat([modelos_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(modelos_resultados)

#### KNN

In [ ]:
#eliminar clases con solo un valor unico
class_counts = df_cal1['nota_03'].value_counts()
classes_to_keep = class_counts[class_counts > 1].index
df_filtered = df_cal1[df_cal1['nota_03'].isin(classes_to_keep)]
X = df_cal3[df_cal1['nota_03'].isin(classes_to_keep)]
y = df_filtered['nota_03']

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=123, stratify=y)

test_scores = []
train_scores = []

for i in range(1,15):

    knn = KNeighborsClassifier(i)
    knn.fit(X_train,y_train)

    train_scores.append(knn.score(X_train,y_train))
    test_scores.append(knn.score(X_test,y_test))

In [ ]:
max_test_score = max(test_scores)
test_scores_ind = [i for i, v in enumerate(test_scores) if v == max_test_score]
print('Max test score {} % and k = {}'.format(max_test_score*100,list(map(lambda x: x+1, test_scores_ind))))

In [ ]:
knn = KNeighborsClassifier(8)
knn.fit(X_train,y_train)
knn.score(X_test,y_test)

In [ ]:
DF = 'df_cal, 51var'
Modelo = 'KNN_test'
y_pred = knn.predict(X_test)
MSE = mean_squared_error(y_test, y_pred)
print("MSE: %.2f" % MSE)
RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE %.2f" % RMSE)
MAE = mean_absolute_error(y_test, y_pred)
print("MAE: %.2f" % MAE)
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE: %.2f" % MAPE)
r2 = r2_score(y_test, y_pred)
print("R2: %.2f" % r2)
# R2 ajustado
adj_r2 = 1 - (1-r2)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
print("R2-adjusted: %.2f" % adj_r2)

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [Accuracy],
    'MSE': [MSE],
    'RMSE': [RMSE],
    'MAE': [MAE],
    'MAPE': [MAPE],
    'R2': [r2],
    'R2_adjusted': [adj_r2]
})

# Concatenar con el DataFrame original
modelos_resultados = pd.concat([modelos_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(modelos_resultados)

In [ ]:
DF = 'df_cal, 51var'
Modelo = 'KNN_train'
y_pred = knn.predict(X_train)
MSE = mean_squared_error(y_train, y_pred)
print("MSE: %.2f" % MSE)
RMSE = math.sqrt(mean_squared_error(y_train, y_pred))
print("RMSE %.2f" % RMSE)
MAE = mean_absolute_error(y_train, y_pred)
print("MAE: %.2f" % MAE)
MAPE = mean_absolute_percentage_error(y_train, y_pred)
print("MAPE: %.2f" % MAPE)
r2 = r2_score(y_train, y_pred)
print("R2: %.2f" % r2)
# R2 ajustado
adj_r2 = 1 - (1-r2)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
print("R2-adjusted: %.2f" % adj_r2)

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [Accuracy],
    'MSE': [MSE],
    'RMSE': [RMSE],
    'MAE': [MAE],
    'MAPE': [MAPE],
    'R2': [r2],
    'R2_adjusted': [adj_r2]
})

# Concatenar con el DataFrame original
modelos_resultados = pd.concat([modelos_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(modelos_resultados)

#### XGBoost

In [ ]:
X, y = df_cal3, df_cal1['nota_03']
data = load_iris()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

# Crear el modelo de regresión
bst = XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.1, objective='reg:squarederror')

# Entrenar el modelo
bst.fit(X_train, y_train)

In [ ]:
DF = 'df_cal, 51var'
Modelo = 'XGBoost_test'
y_pred = bst.predict(X_test)
Accuracy = np.NaN
print("Accuracy:", Accuracy)
MSE = mean_squared_error(y_test, y_pred)
print("MSE: %.2f" % MSE)
RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE %.2f" % RMSE)
MAE = mean_absolute_error(y_test, y_pred)
print("MAE: %.2f" % MAE)
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE: %.2f" % MAPE)
r2 = r2_score(y_test, y_pred)
print("R2: %.2f" % r2)
# R2 ajustado
adj_r2 = 1 - (1-r2)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
print("R2-adjusted: %.2f" % adj_r2)

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [Accuracy],
    'MSE': [MSE],
    'RMSE': [RMSE],
    'MAE': [MAE],
    'MAPE': [MAPE],
    'R2': [r2],
    'R2_adjusted': [adj_r2]
})

# Concatenar con el DataFrame original
modelos_resultados = pd.concat([modelos_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(modelos_resultados)

In [ ]:
DF = 'df_cal, 51var'
Modelo = 'XGBoost_train'
y_pred = bst.predict(X_train)
Accuracy = np.NaN
print("Accuracy:", Accuracy)
MSE = mean_squared_error(y_train, y_pred)
print("MSE: %.2f" % MSE)
RMSE = math.sqrt(mean_squared_error(y_train, y_pred))
print("RMSE %.2f" % RMSE)
MAE = mean_absolute_error(y_train, y_pred)
print("MAE: %.2f" % MAE)
MAPE = mean_absolute_percentage_error(y_train, y_pred)
print("MAPE: %.2f" % MAPE)
r2 = r2_score(y_train, y_pred)
print("R2: %.2f" % r2)
# R2 ajustado
adj_r2 = 1 - (1-r2)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
print("R2-adjusted: %.2f" % adj_r2)

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [Accuracy],
    'MSE': [MSE],
    'RMSE': [RMSE],
    'MAE': [MAE],
    'MAPE': [MAPE],
    'R2': [r2],
    'R2_adjusted': [adj_r2]
})

# Concatenar con el DataFrame original
modelos_resultados = pd.concat([modelos_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(modelos_resultados)

#### SVR

In [ ]:
X, y = df_cal3, df_cal1['nota_03']
data = load_iris()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

model = SVR(kernel='linear')
model.fit(X_train, y_train)

In [ ]:
DF = 'df_cal, 51var'
Modelo = 'SVR_lineal_test'
y_pred = model.predict(X_test)
Accuracy = np.NaN
print("Accuracy:", Accuracy)
MSE = mean_squared_error(y_test, y_pred)
print("MSE: %.2f" % MSE)
RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE %.2f" % RMSE)
MAE = mean_absolute_error(y_test, y_pred)
print("MAE: %.2f" % MAE)
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE: %.2f" % MAPE)
r2 = r2_score(y_test, y_pred)
print("R2: %.2f" % r2)
# R2 ajustado
adj_r2 = 1 - (1-r2)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
print("R2-adjusted: %.2f" % adj_r2)

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [Accuracy],
    'MSE': [MSE],
    'RMSE': [RMSE],
    'MAE': [MAE],
    'MAPE': [MAPE],
    'R2': [r2],
    'R2_adjusted': [adj_r2]
})

# Concatenar con el DataFrame original
modelos_resultados = pd.concat([modelos_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(modelos_resultados)

In [ ]:
DF = 'df_cal, 51var'
Modelo = 'SVR_lineal_train'
y_pred = model.predict(X_train)
Accuracy = np.NaN
print("Accuracy:", Accuracy)
MSE = mean_squared_error(y_train, y_pred)
print("MSE: %.2f" % MSE)
RMSE = math.sqrt(mean_squared_error(y_train, y_pred))
print("RMSE %.2f" % RMSE)
MAE = mean_absolute_error(y_train, y_pred)
print("MAE: %.2f" % MAE)
MAPE = mean_absolute_percentage_error(y_train, y_pred)
print("MAPE: %.2f" % MAPE)
r2 = r2_score(y_train, y_pred)
print("R2: %.2f" % r2)
# R2 ajustado
adj_r2 = 1 - (1-r2)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
print("R2-adjusted: %.2f" % adj_r2)

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [Accuracy],
    'MSE': [MSE],
    'RMSE': [RMSE],
    'MAE': [MAE],
    'MAPE': [MAPE],
    'R2': [r2],
    'R2_adjusted': [adj_r2]
})

# Concatenar con el DataFrame original
modelos_resultados = pd.concat([modelos_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(modelos_resultados)

In [ ]:
print(df_resultados)

A pesar de haber aplicado metodos de seleccion de variables inicialmente para el modelo de regresion lineal, estas opciones son las que tienen metricas peores. Por el contrario los nuevos modelos, desarrollados con la version del df_cal de 51 variables junto con f regression (df_cal3) tienen metricas en su mayoria que superan a estos anteriores.

De entre estos, el arbol de decisión aunque no es precisamente el que menos error tiene, su r2 y r2 ajustado son bastante altos y el GAP entre ellos es casi nulo.

## ÁLGEBRA VECTORIAL

### Selección de variables métodos de filtrado

#### Selección univariante

##### Método f_classif



> El método f_classif se usa para evaluar la relación entre las características y la variable objetivo categórica, ayudando en la selección de características relevantes. Un alto valor F y un bajo p-valor indican que la característica es significativa.


###### Df_alg1 (Data frame con 38 columnas)

In [ ]:
df_alg1.columns

In [ ]:
x = df_alg1.drop(columns = ['nota_03'], axis=1)
y = df_alg1['nota_03']
x_new, scores, pvalues = select_kbest(df_alg1, y, f_classif, 25) # Obtener columnas seleccionadas
df_alg3 = df_alg1.iloc[:,x_new] # Nuevo conjunto de datos
df_alg3.head() #se escogen los valores p mas pequeños, y las variables tiene relacion con la variable objetivo

>Estas son las 25 columnas que el modelo selecciona para trabajar en las predicciones.

In [ ]:
df_alg3['nota_03'].unique()

In [ ]:
X_alg = df_alg3.drop(columns = ['nota_03'], axis=1)
y_alg = df_alg3["nota_03"]

# Separar los datos en conjuntos de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(X_alg, y_alg, test_size=0.3, random_state=113)

model = LogisticRegression() # definir el modelo
model.fit(x_train, y_train) # entrenar el modelo

y_pred_train = model.predict(x_train) # guardar la predicción para train
y_pred_test = model.predict(x_test) # guardar la predicción para test

In [ ]:
# Matriz de confusión:
cm = confusion_matrix(y_test, y_pred_test, labels=model.classes_) # guardar las clases para la matriz de confusión
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=model.classes_)
disp.plot();
print(cm)

In [ ]:
TP=cm[1,1] # El modelo dijo que eran 1 y en la realidad son 1, verdaderos positivos - 89
FP=cm[0,1] # El modelo dijo que eran 1, en la realidad son 0, falsos positivos - 19
FN=cm[1,0] # El modelo dijo que eran 0, en la realidad son 1, falsos negativos - 6
TN=cm[0,0] # El modelo dijo que eran 0 y en la realidad son 0, verdaderos negativos - 81

DF = 'df_alg, 38var'
Modelo = 'f_classif_train'

# Métricas de clasificación para el conjunto de entrenamiento
Accuracy = accuracy_score(y_train, y_pred_train)
print("Accuracy:", Accuracy)

# Cálculo de las métricas
Precision = TP / (TP + FP) if (TP + FP) > 0 else np.NaN
Recall = TP / (TP + FN) if (TP + FN) > 0 else np.NaN
F1_score = f1_score(y_train, y_pred_train, average='binary')
Especificidad = TN / (FP + TN) if (FP + TN) > 0 else np.NaN

# Imprimir métricas
print(f'Precisión: {Precision}')
print(f'Recall (Sensibilidad): {Recall}')
print(f'F1-score: {F1_score}')
print(f'Especificidad: {Especificidad}')

In [ ]:
alg_resultados = pd.DataFrame(columns=['DF', 'Modelo', 'Accuracy', 'Precisión', 'Recall', 'F1-score', 'Especificidad'])

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [Accuracy],
    'Precisión': [Precision],
    'Recall': [Recall],
    'F1-score': [F1_score],
    'Especificidad': [Especificidad]
})

# Concatenar con el DataFrame original
alg_resultados = pd.concat([alg_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(alg_resultados)

In [ ]:
# Metricas de entrenamiento
cm = confusion_matrix(y_test, y_pred_test, labels=model.classes_) # guardar las clases para la matriz de confusión

TP=cm[1,1] # El modelo dijo que eran 1 y en la realidad son 1, verdaderos positivos - 89
FP=cm[0,1] # El modelo dijo que eran 1, en la realidad son 0, falsos positivos - 19
FN=cm[1,0] # El modelo dijo que eran 0, en la realidad son 1, falsos negativos - 6
TN=cm[0,0] # El modelo dijo que eran 0 y en la realidad son 0, verdaderos negativos - 91

TP=cm[1,1] # El modelo dijo que eran 1 y en la realidad son 1, verdaderos positivos - 89
FP=cm[0,1] # El modelo dijo que eran 1, en la realidad son 0, falsos positivos - 19
FN=cm[1,0] # El modelo dijo que eran 0, en la realidad son 1, falsos negativos - 6
TN=cm[0,0] # El modelo dijo que eran 0 y en la realidad son 0, verdaderos negativos - 81

DF = 'df_alg, 38var'
Modelo = 'f_classif_test'

# Métricas de clasificación para el conjunto de entrenamiento
Accuracy = accuracy_score(y_test, y_pred_test)
print("Accuracy:", Accuracy)

# Cálculo de las métricas
Precision = TP / (TP + FP) if (TP + FP) > 0 else np.NaN
Recall = TP / (TP + FN) if (TP + FN) > 0 else np.NaN
F1_score = f1_score(y_test, y_pred_test, average='binary')
Especificidad = TN / (FP + TN) if (FP + TN) > 0 else np.NaN

# Imprimir métricas
print(f'Precisión: {Precision}')
print(f'Recall (Sensibilidad): {Recall}')
print(f'F1-score: {F1_score}')
print(f'Especificidad: {Especificidad}')

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [Accuracy],
    'Precisión': [Precision],
    'Recall': [Recall],
    'F1-score': [F1_score],
    'Especificidad': [Especificidad]
})

# Concatenar con el DataFrame original
alg_resultados = pd.concat([alg_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(alg_resultados)

###### Df_alg2 (Data frame con 31 columnas)

In [ ]:
df_alg2.columns

In [ ]:
x = df_alg2.drop(columns = ['nota_03'], axis=1)
y = df_alg2['nota_03']
x_new, scores, pvalues = select_kbest(df_alg2, y, f_classif, 25) # Obtener columnas seleccionadas
df_alg4 = df_alg2.iloc[:,x_new] # Nuevo conjunto de datos
df_alg4.head() #se escogen los valores p mas pequeños, y las variables tiene relacion con la variable objetivo

>Estas son las 25 columnas que el modelo selecciona para trabajar en las predicciones.

In [ ]:
df_alg4['nota_03'].unique()

In [ ]:
df_alg4['nota_03'] = df_alg4['nota_03'].apply(Transf)

In [ ]:
X_alg = df_alg4.drop(columns = ['nota_03'], axis=1)
y_alg = df_alg4["nota_03"]

# Separar los datos en conjuntos de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(X_alg, y_alg, test_size=0.3, random_state=113)

model = LogisticRegression() # definir el modelo
model.fit(x_train, y_train) # entrenar el modelo

y_pred_train = model.predict(x_train) # guardar la predicción para train
y_pred_test = model.predict(x_test) # guardar la predicción para test

In [ ]:
# Matriz de confusión:
cm = confusion_matrix(y_test, y_pred_test, labels=model.classes_) # guardar las clases para la matriz de confusión
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=model.classes_)
disp.plot();
print(cm)

In [ ]:
TP=cm[1,1] # El modelo dijo que eran 1 y en la realidad son 1, verdaderos positivos - 89
FP=cm[0,1] # El modelo dijo que eran 1, en la realidad son 0, falsos positivos - 19
FN=cm[1,0] # El modelo dijo que eran 0, en la realidad son 1, falsos negativos - 6
TN=cm[0,0] # El modelo dijo que eran 0 y en la realidad son 0, verdaderos negativos - 81

DF = 'df_alg, 31var'
Modelo = 'f_classif_train'

# Métricas de clasificación para el conjunto de entrenamiento
Accuracy = accuracy_score(y_train, y_pred_train)
print("Accuracy:", Accuracy)

# Cálculo de las métricas
Precision = TP / (TP + FP) if (TP + FP) > 0 else np.NaN
Recall = TP / (TP + FN) if (TP + FN) > 0 else np.NaN
F1_score = f1_score(y_train, y_pred_train, average='binary')
Especificidad = TN / (FP + TN) if (FP + TN) > 0 else np.NaN

# Imprimir métricas
print(f'Precisión: {Precision}')
print(f'Recall (Sensibilidad): {Recall}')
print(f'F1-score: {F1_score}')
print(f'Especificidad: {Especificidad}')

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [Accuracy],
    'Precisión': [Precision],
    'Recall': [Recall],
    'F1-score': [F1_score],
    'Especificidad': [Especificidad]
})

# Concatenar con el DataFrame original
alg_resultados = pd.concat([alg_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(alg_resultados)

In [ ]:
# Metricas de entrenamiento
cm = confusion_matrix(y_test, y_pred_test, labels=model.classes_) # guardar las clases para la matriz de confusión

TP=cm[1,1] # El modelo dijo que eran 1 y en la realidad son 1, verdaderos positivos - 89
FP=cm[0,1] # El modelo dijo que eran 1, en la realidad son 0, falsos positivos - 19
FN=cm[1,0] # El modelo dijo que eran 0, en la realidad son 1, falsos negativos - 6
TN=cm[0,0] # El modelo dijo que eran 0 y en la realidad son 0, verdaderos negativos - 91

TP=cm[1,1] # El modelo dijo que eran 1 y en la realidad son 1, verdaderos positivos - 89
FP=cm[0,1] # El modelo dijo que eran 1, en la realidad son 0, falsos positivos - 19
FN=cm[1,0] # El modelo dijo que eran 0, en la realidad son 1, falsos negativos - 6
TN=cm[0,0] # El modelo dijo que eran 0 y en la realidad son 0, verdaderos negativos - 81

DF = 'df_alg, 31var'
Modelo = 'f_classif_test'

# Métricas de clasificación para el conjunto de entrenamiento
Accuracy = accuracy_score(y_test, y_pred_test)
print("Accuracy:", Accuracy)

# Cálculo de las métricas
Precision = TP / (TP + FP) if (TP + FP) > 0 else np.NaN
Recall = TP / (TP + FN) if (TP + FN) > 0 else np.NaN
F1_score = f1_score(y_test, y_pred_test, average='binary')
Especificidad = TN / (FP + TN) if (FP + TN) > 0 else np.NaN

# Imprimir métricas
print(f'Precisión: {Precision}')
print(f'Recall (Sensibilidad): {Recall}')
print(f'F1-score: {F1_score}')
print(f'Especificidad: {Especificidad}')

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [Accuracy],
    'Precisión': [Precision],
    'Recall': [Recall],
    'F1-score': [F1_score],
    'Especificidad': [Especificidad]
})

# Concatenar con el DataFrame original
alg_resultados = pd.concat([alg_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(alg_resultados)

##### Método mutual info regression

> Este método estima cuánta información sobre la variable objetivo se puede obtener de cada característica, siendo útil para la selección de características

###### Df_alg1 (Data frame con 38 columnas)

In [ ]:
y = df_alg1['nota_03']
x_new, scores, pvalues = select_kbest(df_alg1, y, mutual_info_regression, 31) # Obtener columnas seleciconadas
df_alg4 = df_alg1.iloc[:,x_new] # Nuevo conjunto de datos
df_alg4.head() #se escogen los valores p mas pequeños, y las variables tiene relacion con la variable objetivo

In [ ]:
df_alg4['nota_03'].unique()

In [ ]:
X_alg = df_alg4.drop("nota_03", axis=1)
y_alg = df_alg4["nota_03"]

# Separar los datos en conjuntos de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(X_alg, y_alg, test_size=0.3, random_state=113)

model = LogisticRegression() # definir el modelo
model.fit(x_train, y_train) # entrenar el modelo

y_pred_train = model.predict(x_train) # guardar la predicción para train
y_pred_test = model.predict(x_test) # guardar la predicción para test

In [ ]:
# Matriz de confusión:
cm = confusion_matrix(y_test, y_pred_test, labels=model.classes_) # guardar las clases para la matriz de confusión
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=model.classes_)
disp.plot();
print(cm)

In [ ]:
TP=cm[1,1] # El modelo dijo que eran 1 y en la realidad son 1, verdaderos positivos - 89
FP=cm[0,1] # El modelo dijo que eran 1, en la realidad son 0, falsos positivos - 19
FN=cm[1,0] # El modelo dijo que eran 0, en la realidad son 1, falsos negativos - 6
TN=cm[0,0] # El modelo dijo que eran 0 y en la realidad son 0, verdaderos negativos - 81

DF = 'df_alg, 38var'
Modelo = 'f_mutual_info_train'

# Métricas de clasificación para el conjunto de entrenamiento
Accuracy = accuracy_score(y_train, y_pred_train)
print("Accuracy:", Accuracy)

# Cálculo de las métricas
Precision = TP / (TP + FP) if (TP + FP) > 0 else np.NaN
Recall = TP / (TP + FN) if (TP + FN) > 0 else np.NaN
F1_score = f1_score(y_train, y_pred_train, average='binary')
Especificidad = TN / (FP + TN) if (FP + TN) > 0 else np.NaN

# Imprimir métricas
print(f'Precisión: {Precision}')
print(f'Recall (Sensibilidad): {Recall}')
print(f'F1-score: {F1_score}')
print(f'Especificidad: {Especificidad}')

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [Accuracy],
    'Precisión': [Precision],
    'Recall': [Recall],
    'F1-score': [F1_score],
    'Especificidad': [Especificidad]
})

# Concatenar con el DataFrame original
alg_resultados = pd.concat([alg_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(alg_resultados)

In [ ]:
# Metricas de entrenamiento
cm = confusion_matrix(y_test, y_pred_test, labels=model.classes_) # guardar las clases para la matriz de confusión

TP=cm[1,1] # El modelo dijo que eran 1 y en la realidad son 1, verdaderos positivos - 89
FP=cm[0,1] # El modelo dijo que eran 1, en la realidad son 0, falsos positivos - 19
FN=cm[1,0] # El modelo dijo que eran 0, en la realidad son 1, falsos negativos - 6
TN=cm[0,0] # El modelo dijo que eran 0 y en la realidad son 0, verdaderos negativos - 91

TP=cm[1,1] # El modelo dijo que eran 1 y en la realidad son 1, verdaderos positivos - 89
FP=cm[0,1] # El modelo dijo que eran 1, en la realidad son 0, falsos positivos - 19
FN=cm[1,0] # El modelo dijo que eran 0, en la realidad son 1, falsos negativos - 6
TN=cm[0,0] # El modelo dijo que eran 0 y en la realidad son 0, verdaderos negativos - 81

DF = 'df_alg, 38var'
Modelo = 'f_mutual_info_test'

# Métricas de clasificación para el conjunto de entrenamiento
Accuracy = accuracy_score(y_test, y_pred_test)
print("Accuracy:", Accuracy)

# Cálculo de las métricas
Precision = TP / (TP + FP) if (TP + FP) > 0 else np.NaN
Recall = TP / (TP + FN) if (TP + FN) > 0 else np.NaN
F1_score = f1_score(y_test, y_pred_test, average='binary')
Especificidad = TN / (FP + TN) if (FP + TN) > 0 else np.NaN

# Imprimir métricas
print(f'Precisión: {Precision}')
print(f'Recall (Sensibilidad): {Recall}')
print(f'F1-score: {F1_score}')
print(f'Especificidad: {Especificidad}')

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [Accuracy],
    'Precisión': [Precision],
    'Recall': [Recall],
    'F1-score': [F1_score],
    'Especificidad': [Especificidad]
})

# Concatenar con el DataFrame original
alg_resultados = pd.concat([alg_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(alg_resultados)

###### Df_alg2 (Data frame con 31 columnas)

In [ ]:
y = df_alg2['nota_03']
x_new, scores, pvalues = select_kbest(df_alg2, y, mutual_info_regression, 28) # Obtener columnas seleciconadas
df_alg5 = df_alg2.iloc[:,x_new] # Nuevo conjunto de datos
df_alg5.head() #se escogen los valores p mas pequeños, y las variables tiene relacion con la variable objetivo

In [ ]:
df_alg5['nota_03'].unique()

In [ ]:
df_alg5['nota_03'] = df_alg5['nota_03'].apply(Transf)

In [ ]:
X_alg = df_alg5.drop("nota_03", axis=1)
y_alg = df_alg5["nota_03"]

# Separar los datos en conjuntos de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(X_alg, y_alg, test_size=0.3, random_state=113)

model = LogisticRegression() # definir el modelo
model.fit(x_train, y_train) # entrenar el modelo

y_pred_train = model.predict(x_train) # guardar la predicción para train
y_pred_test = model.predict(x_test) # guardar la predicción para test

In [ ]:
# Matriz de confusión:
cm = confusion_matrix(y_test, y_pred_test, labels=model.classes_) # guardar las clases para la matriz de confusión
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=model.classes_)
disp.plot();
print(cm)

In [ ]:
TP=cm[1,1] # El modelo dijo que eran 1 y en la realidad son 1, verdaderos positivos - 89
FP=cm[0,1] # El modelo dijo que eran 1, en la realidad son 0, falsos positivos - 19
FN=cm[1,0] # El modelo dijo que eran 0, en la realidad son 1, falsos negativos - 6
TN=cm[0,0] # El modelo dijo que eran 0 y en la realidad son 0, verdaderos negativos - 81

DF = 'df_alg, 31var'
Modelo = 'f_mutual_info_train'

# Métricas de clasificación para el conjunto de entrenamiento
Accuracy = accuracy_score(y_train, y_pred_train)
print("Accuracy:", Accuracy)

# Cálculo de las métricas
Precision = TP / (TP + FP) if (TP + FP) > 0 else np.NaN
Recall = TP / (TP + FN) if (TP + FN) > 0 else np.NaN
F1_score = f1_score(y_train, y_pred_train, average='binary')
Especificidad = TN / (FP + TN) if (FP + TN) > 0 else np.NaN

# Imprimir métricas
print(f'Precisión: {Precision}')
print(f'Recall (Sensibilidad): {Recall}')
print(f'F1-score: {F1_score}')
print(f'Especificidad: {Especificidad}')

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [Accuracy],
    'Precisión': [Precision],
    'Recall': [Recall],
    'F1-score': [F1_score],
    'Especificidad': [Especificidad]
})

# Concatenar con el DataFrame original
alg_resultados = pd.concat([alg_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(alg_resultados)

In [ ]:
# Metricas de entrenamiento
cm = confusion_matrix(y_test, y_pred_test, labels=model.classes_) # guardar las clases para la matriz de confusión

TP=cm[1,1] # El modelo dijo que eran 1 y en la realidad son 1, verdaderos positivos - 89
FP=cm[0,1] # El modelo dijo que eran 1, en la realidad son 0, falsos positivos - 19
FN=cm[1,0] # El modelo dijo que eran 0, en la realidad son 1, falsos negativos - 6
TN=cm[0,0] # El modelo dijo que eran 0 y en la realidad son 0, verdaderos negativos - 91

TP=cm[1,1] # El modelo dijo que eran 1 y en la realidad son 1, verdaderos positivos - 89
FP=cm[0,1] # El modelo dijo que eran 1, en la realidad son 0, falsos positivos - 19
FN=cm[1,0] # El modelo dijo que eran 0, en la realidad son 1, falsos negativos - 6
TN=cm[0,0] # El modelo dijo que eran 0 y en la realidad son 0, verdaderos negativos - 81

DF = 'df_alg, 31var'
Modelo = 'f_mutual_info_test'

# Métricas de clasificación para el conjunto de entrenamiento
Accuracy = accuracy_score(y_test, y_pred_test)
print("Accuracy:", Accuracy)

# Cálculo de las métricas
Precision = TP / (TP + FP) if (TP + FP) > 0 else np.NaN
Recall = TP / (TP + FN) if (TP + FN) > 0 else np.NaN
F1_score = f1_score(y_test, y_pred_test, average='binary')
Especificidad = TN / (FP + TN) if (FP + TN) > 0 else np.NaN

# Imprimir métricas
print(f'Precisión: {Precision}')
print(f'Recall (Sensibilidad): {Recall}')
print(f'F1-score: {F1_score}')
print(f'Especificidad: {Especificidad}')

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [Accuracy],
    'Precisión': [Precision],
    'Recall': [Recall],
    'F1-score': [F1_score],
    'Especificidad': [Especificidad]
})

# Concatenar con el DataFrame original
alg_resultados = pd.concat([alg_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(alg_resultados)

Por su GAP, preferimos el f_classif de la base con 31 variables, pues generaliza mejor.

### Optimización de hiper-parámetros

#### Regresión Ridge

In [ ]:
#Se realiza una copia del data frame original pre-procesado
df_alg7= df_alg1.copy()
X = df_alg7.drop('nota_03', axis=1)
Y = df_alg7['nota_03']

In [ ]:
from sklearn.linear_model import Ridge, LogisticRegression, LinearRegression
from sklearn.model_selection import GridSearchCV, cross_val_score

RidgeRegression = Ridge()
hyperParameters = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40,45,50,55,100]}
ridgeRegressor = GridSearchCV(RidgeRegression, hyperParameters, scoring='neg_mean_squared_error')
ridgeRegressor.fit(X,y)

In [ ]:
print("Best value for lambda : ",ridgeRegressor.best_params_)
print("Best score for cost function: ", ridgeRegressor.best_score_)

#### Regresión Lasso

In [ ]:
from sklearn.linear_model import Lasso

LassoRegression = Lasso()
hyperParameters = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40,45,50,55,100]}
LassoRegressor = GridSearchCV(LassoRegression, hyperParameters, scoring='neg_mean_squared_error')
LassoRegressor.fit(X,y)

In [ ]:
print("Best value for lambda : ",LassoRegressor.best_params_)
print("Best score for cost function: ", LassoRegressor.best_score_)

#### Comparativa

In [ ]:
# Definir función
from sklearn.model_selection import train_test_split
def test(models, X, y, iterations = 100):
    results = {}
    for i in models:
        mse_train = []
        mse_test = []
        for j in range(iterations):
            X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                                test_size= 0.2)
            mse_test.append(metrics.mean_squared_error(y_test,
                                            models[i].fit(X_train,
                                                         y_train).predict(X_test)))
            mse_train.append(metrics.mean_squared_error(y_train,
                                             models[i].fit(X_train,
                                                          y_train).predict(X_train)))
        results[i] = [np.mean(mse_train), np.mean(mse_test)]
    return pd.DataFrame(results)

In [ ]:
models = {'OLS': LinearRegression(),
         'Lasso': Lasso(),
         'Ridge': Ridge(),}

In [ ]:
pd.options.display.float_format = '{:.5f}'.format
test(models, X, y)

In [ ]:
lasso_params = {'alpha':[0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03]}
ridge_params = {'alpha':[0.01, 0.012, 0.014, 0.016, 0.018, 0.02, 0.022, 0.024, 0.026, 0.028, 0.03]}

models2 = {'OLS': LinearRegression(),
           'Lasso': GridSearchCV(Lasso(),
                               param_grid=lasso_params).fit(X, y).best_estimator_,
           'Ridge': GridSearchCV(Ridge(),
                               param_grid=ridge_params).fit(X, y).best_estimator_,}

In [ ]:
test(models2, X, y)

### Otros modelos

#### Arboles de deción


In [ ]:
# Dividir en independientes y dependiente
X, y = df_alg3.drop('nota_03', axis=1), df_alg3['nota_03']

# Definir modelo
clf = tree.DecisionTreeClassifier(max_depth=5)  # Ajusta el max_depth según sea necesario

# Dividir en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 123, test_size= 0.2)

# Entrenar modelo
clf = clf.fit(X_train, y_train)

In [ ]:
# Visualizar el árbol
dot_data = tree.export_graphviz(clf,
                                feature_names=df_alg3.drop('nota_03', axis=1).columns,
                                class_names=list(map(str, df_alg3['nota_03'].unique())),
                                filled=True, rounded=True,
                                special_characters=True)
graph = graphviz.Source(dot_data)
graph.format = 'png'
graph.render('decision_tree', view=True)

# Mostrar la imagen
Image("decision_tree.png")

In [ ]:
# Hacer predicciones en el conjunto de entrenamiento
y_pred_train = clf.predict(X_train)

# Calcular la matriz de confusión
cm = confusion_matrix(y_train, y_pred_train)

# Extraer valores de la matriz de confusión
TP = cm[1, 1]  # Verdaderos Positivos
FP = cm[0, 1]  # Falsos Positivos
FN = cm[1, 0]  # Falsos Negativos
TN = cm[0, 0]  # Verdaderos Negativos

DF = 'df_alg, 31var'
Modelo = 'arboles_decision_train'

# Métricas de clasificación para el conjunto de entrenamiento
Accuracy = accuracy_score(y_train, y_pred_train)
print("Accuracy:", Accuracy)

# Cálculo de las métricas
Precision = TP / (TP + FP) if (TP + FP) > 0 else np.NaN
Recall = TP / (TP + FN) if (TP + FN) > 0 else np.NaN
F1_score_value = f1_score(y_train, y_pred_train, average='binary')
Especificidad = TN / (FP + TN) if (FP + TN) > 0 else np.NaN

# Imprimir métricas
print(f'Precisión: {Precision}')
print(f'Recall (Sensibilidad): {Recall}')
print(f'F1-score: {F1_score_value}')
print(f'Especificidad: {Especificidad}')

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [Accuracy],
    'Precisión': [Precision],
    'Recall': [Recall],
    'F1-score': [F1_score],
    'Especificidad': [Especificidad]
})

# Concatenar con el DataFrame original
alg_resultados = pd.concat([alg_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(alg_resultados)

In [ ]:
# Hacer predicciones en el conjunto de entrenamiento
y_pred_test = clf.predict(X_test)

# Calcular la matriz de confusión
cm = confusion_matrix(y_test, y_pred_test)

# Extraer valores de la matriz de confusión
TP = cm[1, 1]  # Verdaderos Positivos
FP = cm[0, 1]  # Falsos Positivos
FN = cm[1, 0]  # Falsos Negativos
TN = cm[0, 0]  # Verdaderos Negativos

DF = 'df_alg, 31var'
Modelo = 'arboles_decision_test'

# Métricas de clasificación para el conjunto de entrenamiento
Accuracy = accuracy_score(y_test, y_pred_test)
print("Accuracy:", Accuracy)

# Cálculo de las métricas
Precision = TP / (TP + FP) if (TP + FP) > 0 else np.NaN
Recall = TP / (TP + FN) if (TP + FN) > 0 else np.NaN
F1_score_value = f1_score(y_test, y_pred_test, average='binary')
Especificidad = TN / (FP + TN) if (FP + TN) > 0 else np.NaN

# Imprimir métricas
print(f'Precisión: {Precision}')
print(f'Recall (Sensibilidad): {Recall}')
print(f'F1-score: {F1_score_value}')
print(f'Especificidad: {Especificidad}')

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [Accuracy],
    'Precisión': [Precision],
    'Recall': [Recall],
    'F1-score': [F1_score],
    'Especificidad': [Especificidad]
})

# Concatenar con el DataFrame original
alg_resultados = pd.concat([alg_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(alg_resultados)

#### Bosques Aleatorios

In [ ]:
model = RandomForestClassifier(n_estimators = 100, # número de arboles
                               criterion = 'gini',  #Se refiere al error cuadrático medio.
                               max_depth = 4, # estrategia de parada
                               max_leaf_nodes = 10, # estrategia de parada
                               max_features = None,
                               oob_score = False,
                               n_jobs = -1,
                               random_state = 123)
model.fit(X_train, y_train)

In [ ]:
# Hacer predicciones en el conjunto de entrenamiento
y_pred_train = model.predict(X_train)

# Calcular la matriz de confusión
cm = confusion_matrix(y_train, y_pred_train)

# Extraer valores de la matriz de confusión
TP = cm[1, 1]  # Verdaderos Positivos
FP = cm[0, 1]  # Falsos Positivos
FN = cm[1, 0]  # Falsos Negativos
TN = cm[0, 0]  # Verdaderos Negativos

DF = 'df_alg, 31var'
Modelo = 'bosques_aleatorios_train'

# Métricas de clasificación para el conjunto de entrenamiento
Accuracy = accuracy_score(y_train, y_pred_train)
print("Accuracy:", Accuracy)

# Cálculo de las métricas
Precision = TP / (TP + FP) if (TP + FP) > 0 else np.NaN
Recall = TP / (TP + FN) if (TP + FN) > 0 else np.NaN
F1_score_value = f1_score(y_train, y_pred_train, average='binary')
Especificidad = TN / (FP + TN) if (FP + TN) > 0 else np.NaN

# Imprimir métricas
print(f'Precisión: {Precision}')
print(f'Recall (Sensibilidad): {Recall}')
print(f'F1-score: {F1_score_value}')
print(f'Especificidad: {Especificidad}')

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [Accuracy],
    'Precisión': [Precision],
    'Recall': [Recall],
    'F1-score': [F1_score],
    'Especificidad': [Especificidad]
})

# Concatenar con el DataFrame original
alg_resultados = pd.concat([alg_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(alg_resultados)

In [ ]:
# Hacer predicciones en el conjunto de entrenamiento
y_pred_test = model.predict(X_test)

# Calcular la matriz de confusión
cm = confusion_matrix(y_test, y_pred_test)

# Extraer valores de la matriz de confusión
TP = cm[1, 1]  # Verdaderos Positivos
FP = cm[0, 1]  # Falsos Positivos
FN = cm[1, 0]  # Falsos Negativos
TN = cm[0, 0]  # Verdaderos Negativos

DF = 'df_alg, 31var'
Modelo = 'bosques_aleatorios_test'

# Métricas de clasificación para el conjunto de entrenamiento
Accuracy = accuracy_score(y_test, y_pred_test)
print("Accuracy:", Accuracy)

# Cálculo de las métricas
Precision = TP / (TP + FP) if (TP + FP) > 0 else np.NaN
Recall = TP / (TP + FN) if (TP + FN) > 0 else np.NaN
F1_score_value = f1_score(y_test, y_pred_test, average='binary')
Especificidad = TN / (FP + TN) if (FP + TN) > 0 else np.NaN

# Imprimir métricas
print(f'Precisión: {Precision}')
print(f'Recall (Sensibilidad): {Recall}')
print(f'F1-score: {F1_score_value}')
print(f'Especificidad: {Especificidad}')

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [Accuracy],
    'Precisión': [Precision],
    'Recall': [Recall],
    'F1-score': [F1_score],
    'Especificidad': [Especificidad]
})

# Concatenar con el DataFrame original
alg_resultados = pd.concat([alg_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(alg_resultados)

#### Gradient Boosting

In [ ]:
model = GradientBoostingClassifier()

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

model.fit(X_train, y_train)

In [ ]:
# Hacer predicciones en el conjunto de entrenamiento
y_pred_train = model.predict(X_train)

# Calcular la matriz de confusión
cm = confusion_matrix(y_train, y_pred_train)

# Extraer valores de la matriz de confusión
TP = cm[1, 1]  # Verdaderos Positivos
FP = cm[0, 1]  # Falsos Positivos
FN = cm[1, 0]  # Falsos Negativos
TN = cm[0, 0]  # Verdaderos Negativos

DF = 'df_alg, 31var'
Modelo = 'gardient_boosting_train'

# Métricas de clasificación para el conjunto de entrenamiento
Accuracy = accuracy_score(y_train, y_pred_train)
print("Accuracy:", Accuracy)

# Cálculo de las métricas
Precision = TP / (TP + FP) if (TP + FP) > 0 else np.NaN
Recall = TP / (TP + FN) if (TP + FN) > 0 else np.NaN
F1_score_value = f1_score(y_train, y_pred_train, average='binary')
Especificidad = TN / (FP + TN) if (FP + TN) > 0 else np.NaN

# Imprimir métricas
print(f'Precisión: {Precision}')
print(f'Recall (Sensibilidad): {Recall}')
print(f'F1-score: {F1_score_value}')
print(f'Especificidad: {Especificidad}')

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [Accuracy],
    'Precisión': [Precision],
    'Recall': [Recall],
    'F1-score': [F1_score],
    'Especificidad': [Especificidad]
})

# Concatenar con el DataFrame original
alg_resultados = pd.concat([alg_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(alg_resultados)

In [ ]:
# Hacer predicciones en el conjunto de entrenamiento
y_pred_test = model.predict(X_test)

# Calcular la matriz de confusión
cm = confusion_matrix(y_test, y_pred_test)

# Extraer valores de la matriz de confusión
TP = cm[1, 1]  # Verdaderos Positivos
FP = cm[0, 1]  # Falsos Positivos
FN = cm[1, 0]  # Falsos Negativos
TN = cm[0, 0]  # Verdaderos Negativos

DF = 'df_alg, 31var'
Modelo = 'gardient_boosting_test'

# Métricas de clasificación para el conjunto de entrenamiento
Accuracy = accuracy_score(y_test, y_pred_test)
print("Accuracy:", Accuracy)

# Cálculo de las métricas
Precision = TP / (TP + FP) if (TP + FP) > 0 else np.NaN
Recall = TP / (TP + FN) if (TP + FN) > 0 else np.NaN
F1_score_value = f1_score(y_test, y_pred_test, average='binary')
Especificidad = TN / (FP + TN) if (FP + TN) > 0 else np.NaN

# Imprimir métricas
print(f'Precisión: {Precision}')
print(f'Recall (Sensibilidad): {Recall}')
print(f'F1-score: {F1_score_value}')
print(f'Especificidad: {Especificidad}')

In [ ]:
# Crear una nueva fila como DataFrame
nueva_fila = pd.DataFrame({
    'DF': [DF],
    'Modelo': [Modelo],
    'Accuracy': [Accuracy],
    'Precisión': [Precision],
    'Recall': [Recall],
    'F1-score': [F1_score],
    'Especificidad': [Especificidad]
})

# Concatenar con el DataFrame original
alg_resultados = pd.concat([alg_resultados, nueva_fila], ignore_index=True)

# Mostrar el DataFrame con los resultados
print(alg_resultados)

De los modelos adicionales probados, se nota la gran diferencia en mejoría respecto a la regresión logística. De entre ellos arboles de decisión presenta menor gap por ende es el que mejor generaliza.